<a href="https://colab.research.google.com/github/ijamil1/Machine-Learning-JHU-/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Prep

In [ ]:
import torch
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import skimage.data
import skimage.color
import scipy.misc
import scipy.signal
import time
import torch.nn.functional as F
from matplotlib import rcParams
rcParams['axes.grid'] = False
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
#train_identity = pd.read_csv("train_identity.csv")
train_identity = pd.read_csv("/content/drive/My Drive/Deep Learning Group/ieee-fraud-detection/train_identity.csv") #(Gabe's filepath)
#train_identity = pd.read_csv("/content/drive/My Drive/train_identity.csv") (Irfan's filepath)
#train_identity = pd.read_csv("/content/drive/My Drive/train_identity.csv")
#train_identity = pd.read_csv(io.BytesIO(uploaded['train_identity.csv']))
test_identity = pd.read_csv("/content/drive/My Drive/Deep Learning Group/ieee-fraud-detection/test_identity.csv")

In [ ]:
colNames=['TransactionId','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30',
          '31','32','33','34','35','36','37','38','DeviceType','DeviceInfo']

In [ ]:
train_identity.columns=colNames
test_identity.columns=colNames

In [ ]:
train_identity

,TransactionId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,NaN,427.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,-300.0,New,NotFound,166.0,15.0,352.0,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.1.1,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,27.0,567.0,411.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,NaN,NaN,0.0,0.0,100.0,NotFound,27.0,-300.0,Found,Found,166.0,15.0,352.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_9_5,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [ ]:
nullList=[['TransactionId',0.0]]
for i in range(38):
  j=i+1
  idx=str(j)
  perecentageNull=(train_identity[idx].isnull().sum())/train_identity.shape[0]
  nullList.append([i+1,perecentageNull])


In [ ]:
nullList.append(['DeviceType',train_identity['DeviceType'].isnull().sum()/train_identity.shape[0]])
nullList.append(['DeviceInfo',train_identity['DeviceInfo'].isnull().sum()/train_identity.shape[0]])
nullList

[['TransactionId', 0.0],
 [1, 0.0],
 [2, 0.0233025729202055],
 [3, 0.5401607121809849],
 [4, 0.5401607121809849],
 [5, 0.05108400990064687],
 [6, 0.05108400990064687],
 [7, 0.9642592194574058],
 [8, 0.9642592194574058],
 [9, 0.4805211012736336],
 [10, 0.4805211012736336],
 [11, 0.02256765095366525],
 [12, 0.0],
 [13, 0.11726165301976663],
 [14, 0.4450368500967185],
 [15, 0.022519118370969194],
 [16, 0.10325653629890524],
 [17, 0.033723211747658306],
 [18, 0.6872213709761289],
 [19, 0.034076806278729554],
 [20, 0.034472000166397426],
 [21, 0.9642314865530079],
 [22, 0.9641621542920136],
 [23, 0.9641621542920136],
 [24, 0.9670879757059757],
 [25, 0.9644186836576928],
 [26, 0.9642037536486102],
 [27, 0.9641621542920136],
 [28, 0.02256765095366525],
 [29, 0.02256765095366525],
 [30, 0.46222431759722116],
 [31, 0.027393176318872933],
 [32, 0.462078719849133],
 [33, 0.4918707923984109],
 [34, 0.46056034333335644],
 [35, 0.022519118370969194],
 [36, 0.022519118370969194],
 [37, 0.022519118370

In [ ]:
#so NullList is a list of lists in which the first element is the column name and the second element is the percentage of that column that is null

In [ ]:
#Cols 3,4,7,8,9,10,14,18,21,22,23,24,25,26,27,30,32,33,34 have high null percentages and these columns account for about half of the total number of cols
#There are certain columns which have about 96% null rate so I think it makes sense to delete this columns

In [ ]:
Cols2Delete=[]
for i in range(len(nullList)-3):
  j=i+1
  if (nullList[j][1]>0.9):
    Cols2Delete.append(j)

In [ ]:
Cols2Delete

[7, 8, 21, 22, 23, 24, 25, 26, 27]

In [ ]:
for i in range(len(Cols2Delete)):
  Cols2Delete[i]=str(Cols2Delete[i])
Cols2Delete

['7', '8', '21', '22', '23', '24', '25', '26', '27']

In [ ]:
train_identity=train_identity.drop(columns=Cols2Delete)
test_identity = test_identity.drop(columns=Cols2Delete)
train_identity

,TransactionId,1,2,3,4,5,6,9,10,11,12,13,14,15,16,17,18,19,20,28,29,30,31,32,33,34,35,36,37,38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,NaN,427.0,139.0,Found,Found,NaN,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,100.0,NotFound,27.0,-300.0,New,NotFound,166.0,15.0,352.0,333.0,New,NotFound,Android 7.1.1,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,27.0,567.0,411.0,New,NotFound,NaN,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,0.0,0.0,100.0,NotFound,27.0,-300.0,Found,Found,166.0,15.0,352.0,368.0,Found,Found,Mac OS X 10_9_5,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [ ]:
#Now train_identity is the same dataframe except that columns in which over 90% of the rows had missing data have been removed

In [ ]:
del nullList[7]

In [ ]:
nullList

[['TransactionId', 0.0],
 [1, 0.0],
 [2, 0.0233025729202055],
 [3, 0.5401607121809849],
 [4, 0.5401607121809849],
 [5, 0.05108400990064687],
 [6, 0.05108400990064687],
 [8, 0.9642592194574058],
 [9, 0.4805211012736336],
 [10, 0.4805211012736336],
 [11, 0.02256765095366525],
 [12, 0.0],
 [13, 0.11726165301976663],
 [14, 0.4450368500967185],
 [15, 0.022519118370969194],
 [16, 0.10325653629890524],
 [17, 0.033723211747658306],
 [18, 0.6872213709761289],
 [19, 0.034076806278729554],
 [20, 0.034472000166397426],
 [21, 0.9642314865530079],
 [22, 0.9641621542920136],
 [23, 0.9641621542920136],
 [24, 0.9670879757059757],
 [25, 0.9644186836576928],
 [26, 0.9642037536486102],
 [27, 0.9641621542920136],
 [28, 0.02256765095366525],
 [29, 0.02256765095366525],
 [30, 0.46222431759722116],
 [31, 0.027393176318872933],
 [32, 0.462078719849133],
 [33, 0.4918707923984109],
 [34, 0.46056034333335644],
 [35, 0.022519118370969194],
 [36, 0.022519118370969194],
 [37, 0.022519118370969194],
 [38, 0.022519118

In [ ]:
del nullList[7]

In [ ]:
del nullList[19:26]

In [ ]:
trainIdentitynullList=nullList

In [ ]:
#Now, trainIdentitynullList is updated to reflect the current train_identity table null percentages by column name

In [ ]:
trainIdentitynullList

[['TransactionId', 0.0],
 [1, 0.0],
 [2, 0.0233025729202055],
 [3, 0.5401607121809849],
 [4, 0.5401607121809849],
 [5, 0.05108400990064687],
 [6, 0.05108400990064687],
 [9, 0.4805211012736336],
 [10, 0.4805211012736336],
 [11, 0.02256765095366525],
 [12, 0.0],
 [13, 0.11726165301976663],
 [14, 0.4450368500967185],
 [15, 0.022519118370969194],
 [16, 0.10325653629890524],
 [17, 0.033723211747658306],
 [18, 0.6872213709761289],
 [19, 0.034076806278729554],
 [20, 0.034472000166397426],
 [28, 0.02256765095366525],
 [29, 0.02256765095366525],
 [30, 0.46222431759722116],
 [31, 0.027393176318872933],
 [32, 0.462078719849133],
 [33, 0.4918707923984109],
 [34, 0.46056034333335644],
 [35, 0.022519118370969194],
 [36, 0.022519118370969194],
 [37, 0.022519118370969194],
 [38, 0.022519118370969194],
 ['DeviceType', 0.023732432938370553],
 ['DeviceInfo', 0.17726179168428863]]

In [ ]:
colNames=train_identity.columns

In [ ]:
nonNumericColNames=[]
for i in train_identity.columns:
  j=0
  notFound=True
  idxNull=train_identity[i].isnull()
  while (j<len(train_identity[i])) and notFound:
    if idxNull[j]==False:
      notFound=False
      if (type(train_identity[i][j])==str):
        nonNumericColNames.append(i)
    j+=1
 
 

In [ ]:
nonNumericColNames

['12',
 '15',
 '16',
 '28',
 '29',
 '30',
 '31',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 'DeviceType',
 'DeviceInfo']

In [ ]:
#columns 35-38 are T/F and column DeviceType is either mobile or desktop so we can binary encode them. Let F = -1 and T = 1. Let mobile = -1 and desktop = 1. Recommendations for binary
#encoding follow from James McCaffery (Microsoft Research)

In [ ]:
for i in [35,36,37,38]:
  idxsTrue=(train_identity[str(i)]=='T')
  idxsFalse=(train_identity[str(i)]=='F')
  test_idxsTrue=(test_identity[str(i)]=='T')
  test_idxsFalse=(test_identity[str(i)]=='F')
  train_identity[str(i)][idxsTrue] = 1
  train_identity[str(i)][idxsFalse] = -1
  test_identity[str(i)][test_idxsTrue] = 1
  test_identity[str(i)][test_idxsFalse] = -1



In [ ]:
idxsMobile=(train_identity['DeviceType']=='mobile')
idxsDesktop=(train_identity['DeviceType']=='desktop')
test_idxsMobile=(test_identity['DeviceType']=='mobile')
test_idxsDesktop=(test_identity['DeviceType']=='desktop')
train_identity['DeviceType'][idxsMobile] = -1
train_identity['DeviceType'][idxsDesktop] = 1
test_identity['DeviceType'][test_idxsMobile] = -1
test_identity['DeviceType'][test_idxsDesktop] = 1
train_identity

,TransactionId,1,2,3,4,5,6,9,10,11,12,13,14,15,16,17,18,19,20,28,29,30,31,32,33,34,35,36,37,38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,1,-1,1,1,-1,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,1,-1,-1,1,-1,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,1,-1,1,1,1,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,NaN,427.0,139.0,Found,Found,NaN,chrome 66.0 for android,NaN,NaN,NaN,-1,-1,1,-1,-1,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,100.0,NotFound,27.0,-300.0,New,NotFound,166.0,15.0,352.0,333.0,New,NotFound,Android 7.1.1,chrome 55.0 for android,32.0,855x480,match_status:2,1,-1,1,-1,-1,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,27.0,567.0,411.0,New,NotFound,NaN,chrome 65.0 for android,NaN,NaN,NaN,-1,-1,1,-1,-1,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,0.0,0.0,100.0,NotFound,27.0,-300.0,Found,Found,166.0,15.0,352.0,368.0,Found,Found,Mac OS X 10_9_5,chrome 66.0,24.0,2560x1600,match_status:2,1,-1,1,-1,1,MacOS


In [ ]:
nonNumericColNames.remove('35')


In [ ]:
nonNumericColNames.remove('36')
nonNumericColNames.remove('37')
nonNumericColNames.remove('38')
nonNumericColNames.remove('DeviceType')

In [ ]:
nonNumericColNames

['12', '15', '16', '28', '29', '30', '31', '33', '34', 'DeviceInfo']

In [ ]:
nonNumericColNamesTrainIdentity=nonNumericColNames

In [ ]:
# So, nonNumericColNamesTrainIdentity contains the columns names of those columns in the train_identity dataset that do not have numeric values

In [ ]:
 #train_transaction = pd.read_csv("/content/drive/My Drive/train_transaction.csv")

train_transaction =  pd.read_csv("/content/drive/My Drive/Deep Learning Group/ieee-fraud-detection/train_transaction.csv") #(Gabe's filepath)
test_transaction =  pd.read_csv("/content/drive/My Drive/Deep Learning Group/ieee-fraud-detection/test_transaction.csv")

In [ ]:
train_transaction

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,117.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,50.000000,1758.000000,925.000000,0.0,354.000000,0.0,135.0,0.000000,0.000000,0.000000,50.0,1404.0,790.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,debit,272.0,87.0,48.0,NaN,NaN,NaN,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0,29.0,29.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.0,47.950001,0.0,0.0,47.950001,47.950001,47.950001,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
transactionNullList=[]
for i in train_transaction.columns:
  percentageNull = train_transaction[i].isnull().sum()/train_transaction.shape[0]
  transactionNullList.append([i,perecentageNull])
transactionNullList

[['TransactionID', 0.022519118370969194],
 ['isFraud', 0.022519118370969194],
 ['TransactionDT', 0.022519118370969194],
 ['TransactionAmt', 0.022519118370969194],
 ['ProductCD', 0.022519118370969194],
 ['card1', 0.022519118370969194],
 ['card2', 0.022519118370969194],
 ['card3', 0.022519118370969194],
 ['card4', 0.022519118370969194],
 ['card5', 0.022519118370969194],
 ['card6', 0.022519118370969194],
 ['addr1', 0.022519118370969194],
 ['addr2', 0.022519118370969194],
 ['dist1', 0.022519118370969194],
 ['dist2', 0.022519118370969194],
 ['P_emaildomain', 0.022519118370969194],
 ['R_emaildomain', 0.022519118370969194],
 ['C1', 0.022519118370969194],
 ['C2', 0.022519118370969194],
 ['C3', 0.022519118370969194],
 ['C4', 0.022519118370969194],
 ['C5', 0.022519118370969194],
 ['C6', 0.022519118370969194],
 ['C7', 0.022519118370969194],
 ['C8', 0.022519118370969194],
 ['C9', 0.022519118370969194],
 ['C10', 0.022519118370969194],
 ['C11', 0.022519118370969194],
 ['C12', 0.022519118370969194],


In [ ]:
maxnp = 0.0
for i in range(len(transactionNullList)):
  if (transactionNullList[i][1]>maxnp):
    maxnp=transactionNullList[i][1]
maxnp

0.022519118370969194

In [ ]:
#Thus, we see that the column with the highest percentage of null entries is only 2.2%. So, the train transaction dataset has relatively few missing data points. 

In [ ]:
mergedData=train_identity.merge(train_transaction,left_on=['TransactionId'],right_on=['TransactionID'])
test_mergedData = test_identity.merge(test_transaction,left_on=['TransactionId'],right_on=['TransactionID'])

In [ ]:
mergedData

,TransactionId,1,2,3,4,5,6,9,10,11,12,13,14,15,16,17,18,19,20,28,29,30,31,32,33,34,35,36,37,38,DeviceType,DeviceInfo,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,1,-1,1,1,-1,SAMSUNG SM-G892A Build/NRD90M,2987004,0,86506,50.000,H,4497,514.0,150.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,1,-1,-1,1,-1,iOS Device,2987008,0,86535,15.000,H,2803,100.0,150.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,Windows,2987010,0,86549,75.887,C,16496,352.0,117.0,...,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,90.327904,31.841299,90.327904,90.327904,90.327904,90.327904,0.000000,0.000000,0.000000,75.887497,75.887497,75.887497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,NaN,2987011,0,86555,16.495,C,4461,375.0,185.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,1,-1,1,1,1,MacOS,2987016,0,86620,30.000,H,1790,555.0,150.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,NaN,427.0,139.0,Found,Found,NaN,chrome 66.0 for android,NaN,NaN,NaN,-1,-1,1,-1,-1,F3111 Build/33.3.A.1.97,3577521,0,15810802,48.877,C,12019,305.0,106.0,...,0.0,0.0,2.0,2.0,2.0,1.0,548.831970,548.831970,548.831970,60.066002,60.066002,0.000000,60.066002,60.066002,60.066002,60.066002,488.765991,488.765991,488.765991,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,100.0,NotFound,27.0,-300.0,New,NotFound,166.0,15.0,352.0,333.0,New,NotFound,Android 7.1.1,chrome 55.0 for android,32.0,855x480,match_status:2,1,-1,1,-1,-1,A574BL Build/NMF26F,3577526,1,15810876,250.000,R,1214,174.0,150.0,...,NaN,NaN,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [ ]:
mergedDataNullList=[]
for i in mergedData.columns:
  percentNull = mergedData[i].isnull().sum()/mergedData.shape[0]
  mergedDataNullList.append([i,percentNull])
mergedDataNullList

[['TransactionId', 0.0],
 ['1', 0.0],
 ['2', 0.0233025729202055],
 ['3', 0.5401607121809849],
 ['4', 0.5401607121809849],
 ['5', 0.05108400990064687],
 ['6', 0.05108400990064687],
 ['9', 0.4805211012736336],
 ['10', 0.4805211012736336],
 ['11', 0.02256765095366525],
 ['12', 0.0],
 ['13', 0.11726165301976663],
 ['14', 0.4450368500967185],
 ['15', 0.022519118370969194],
 ['16', 0.10325653629890524],
 ['17', 0.033723211747658306],
 ['18', 0.6872213709761289],
 ['19', 0.034076806278729554],
 ['20', 0.034472000166397426],
 ['28', 0.02256765095366525],
 ['29', 0.02256765095366525],
 ['30', 0.46222431759722116],
 ['31', 0.027393176318872933],
 ['32', 0.462078719849133],
 ['33', 0.4918707923984109],
 ['34', 0.46056034333335644],
 ['35', 0.022519118370969194],
 ['36', 0.022519118370969194],
 ['37', 0.022519118370969194],
 ['38', 0.022519118370969194],
 ['DeviceType', 0.023732432938370553],
 ['DeviceInfo', 0.17726179168428863],
 ['TransactionID', 0.0],
 ['isFraud', 0.0],
 ['TransactionDT', 0.0],

In [ ]:
#So mergedDataNullList is a list of lists in which each list contains two entries with the first being the column name and the second being the percentage of rows
# in the df with missing data. I recalculated this since the number of rows that re in the mergedData dataframe is a subset of the rows in the original transaction dataset
#Thus, the percentage rates of missing data is changed since its conditioned on the row being in the identity dataset as well.

In [ ]:
columns2remove=[]
for i in range(len(mergedDataNullList)):
  colName=mergedDataNullList[i][0]
  if (mergedDataNullList[i][1]>0.75):
    columns2remove.append(colName)
mergedData=mergedData.drop(columns=columns2remove)
test_mergedData = test_mergedData.drop(columns=columns2remove)
mergedData

,TransactionId,1,2,3,4,5,6,9,10,11,12,13,14,15,16,17,18,19,20,28,29,30,31,32,33,34,35,36,37,38,DeviceType,DeviceInfo,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,...,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,1,-1,1,1,-1,SAMSUNG SM-G892A Build/NRD90M,2987004,0,86506,50.000,H,4497,514.0,150.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,1,-1,-1,1,-1,iOS Device,2987008,0,86535,15.000,H,2803,100.0,150.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,Windows,2987010,0,86549,75.887,C,16496,352.0,117.0,...,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,90.327904,31.841299,90.327904,90.327904,90.327904,90.327904,0.000000,0.000000,0.000000,75.887497,75.887497,75.887497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,NaN,2987011,0,86555,16.495,C,4461,375.0,185.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,1,-1,1,1,1,MacOS,2987016,0,86620,30.000,H,1790,555.0,150.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,NaN,427.0,139.0,Found,Found,NaN,chrome 66.0 for android,NaN,NaN,NaN,-1,-1,1,-1,-1,F3111 Build/33.3.A.1.97,3577521,0,15810802,48.877,C,12019,305.0,106.0,...,0.0,0.0,2.0,2.0,2.0,1.0,548.831970,548.831970,548.831970,60.066002,60.066002,0.000000,60.066002,60.066002,60.066002,60.066002,488.765991,488.765991,488.765991,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,100.0,NotFound,27.0,-300.0,New,NotFound,166.0,15.0,352.0,333.0,New,NotFound,Android 7.1.1,chrome 55.0 for android,32.0,855x480,match_status:2,1,-1,1,-1,-1,A574BL Build/NMF26F,3577526,1,15810876,250.000,R,1214,174.0,150.0,...,NaN,NaN,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [ ]:
#So, now mergedData contains only the columns for which the null data accounts for less than 75% of the data for that column. We can continue to throw out or perhaps do imputation on
#other columns with high null rates that are still present in the data. 

In [ ]:
colNamesMergedData=mergedData.columns
colNamesMergedData=list(colNamesMergedData)
isFraudIndex=colNamesMergedData.index('isFraud')
colNamesMergedData = colNamesMergedData[:isFraudIndex] + colNamesMergedData[isFraudIndex+1:]
colNamesMergedData.append('isFraud')


In [ ]:
colNamesMergedData

['TransactionId',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 'DeviceType',
 'DeviceInfo',
 'TransactionID',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D4',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D12',
 'D13',
 'D14',
 'D15',
 'M4',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 

In [ ]:
mergedData=mergedData[colNamesMergedData]

In [ ]:
mergedData

,TransactionId,1,2,3,4,5,6,9,10,11,12,13,14,15,16,17,18,19,20,28,29,30,31,32,33,34,35,36,37,38,DeviceType,DeviceInfo,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,isFraud
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,1,-1,1,1,-1,SAMSUNG SM-G892A Build/NRD90M,2987004,86506,50.000,H,4497,514.0,150.0,mastercard,...,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,1,-1,-1,1,-1,iOS Device,2987008,86535,15.000,H,2803,100.0,150.0,visa,...,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,Windows,2987010,86549,75.887,C,16496,352.0,117.0,mastercard,...,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,90.327904,31.841299,90.327904,90.327904,90.327904,90.327904,0.000000,0.000000,0.000000,75.887497,75.887497,75.887497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,-1,-1,1,1,1,NaN,2987011,86555,16.495,C,4461,375.0,185.0,mastercard,...,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,1,-1,1,1,1,MacOS,2987016,86620,30.000,H,1790,555.0,150.0,visa,...,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,NaN,427.0,139.0,Found,Found,NaN,chrome 66.0 for android,NaN,NaN,NaN,-1,-1,1,-1,-1,F3111 Build/33.3.A.1.97,3577521,15810802,48.877,C,12019,305.0,106.0,mastercard,...,0.0,2.0,2.0,2.0,1.0,548.831970,548.831970,548.831970,60.066002,60.066002,0.000000,60.066002,60.066002,60.066002,60.066002,488.765991,488.765991,488.765991,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,100.0,NotFound,27.0,-300.0,New,NotFound,166.0,15.0,352.0,333.0,New,NotFound,Android 7.1.1,chrome 55.0 for android,32.0,855x480,match_status:2,1,-1,1,-1,-1,A574BL Build/NMF26F,3577526,15810876,250.000,R,1214,174.0,150.0,visa,...,NaN,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0

In [ ]:
#so now merged data is the inner joined dataset of train_identity and train_transaction on 'TransactionId'; columns in which over 75% of the data is missing has been throw out. 
# Also, I reordered the columns so that the 'isFraud' column is the last column in the dataframe. 

In [ ]:
mergedDataNullList.clear()
for i in mergedData.columns:
  percentNull = mergedData[i].isnull().sum()/mergedData.shape[0]
  mergedDataNullList.append([i,percentNull])
mergedDataNullList

[['TransactionId', 0.0],
 ['1', 0.0],
 ['2', 0.0233025729202055],
 ['3', 0.5401607121809849],
 ['4', 0.5401607121809849],
 ['5', 0.05108400990064687],
 ['6', 0.05108400990064687],
 ['9', 0.4805211012736336],
 ['10', 0.4805211012736336],
 ['11', 0.02256765095366525],
 ['12', 0.0],
 ['13', 0.11726165301976663],
 ['14', 0.4450368500967185],
 ['15', 0.022519118370969194],
 ['16', 0.10325653629890524],
 ['17', 0.033723211747658306],
 ['18', 0.6872213709761289],
 ['19', 0.034076806278729554],
 ['20', 0.034472000166397426],
 ['28', 0.02256765095366525],
 ['29', 0.02256765095366525],
 ['30', 0.46222431759722116],
 ['31', 0.027393176318872933],
 ['32', 0.462078719849133],
 ['33', 0.4918707923984109],
 ['34', 0.46056034333335644],
 ['35', 0.022519118370969194],
 ['36', 0.022519118370969194],
 ['37', 0.022519118370969194],
 ['38', 0.022519118370969194],
 ['DeviceType', 0.023732432938370553],
 ['DeviceInfo', 0.17726179168428863],
 ['TransactionID', 0.0],
 ['TransactionDT', 0.0],
 ['TransactionAmt'

In [ ]:
#Now, mergedDataNullList is a list of lists in which the first entry is the column name and the second entry is the percentage of rows in that column in which there is missing data

In [ ]:
mergedDataNonNumericColumns=[]
for i in mergedData.columns:
  firstNonMissingRowIdx=list(mergedData[i].isnull()).index(False)
  if (type(mergedData[i][firstNonMissingRowIdx])==str):
    mergedDataNonNumericColumns.append(i)
mergedDataNonNumericColumns

['12',
 '15',
 '16',
 '28',
 '29',
 '30',
 '31',
 '33',
 '34',
 'DeviceInfo',
 'ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M4']

In [ ]:
#So mergedDataNonNumericColumns tells us exactly which columns in the mergedData dataframe has values that are not numeric.

In [ ]:
set(mergedData['12'])

{'Found', 'NotFound'}

In [ ]:
#column 12 is binary valued with the only 2 possible values being 'Found' and 'Not Found'. Let 'Found' = 1 and 'Not Found' = -1
mergedData['12'][mergedData['12']=='Found'] = 1
mergedData['12'][mergedData['12']=='NotFound']=-1
test_mergedData['12'][test_mergedData['12']=='Found'] = 1
test_mergedData['12'][test_mergedData['12']=='NotFound']=-1

In [ ]:
set(test_mergedData['16'])

{'Found', 'NotFound', nan}

In [ ]:
#column 16 is binary valued with the only 2 non-null values being 'Found' and 'Not Found'. Let 'Found' = 1 and 'Not Found' = -1.
mergedData['16'][mergedData['16']=='Found'] = 1
mergedData['16'][mergedData['16']=='NotFound']=-1
test_mergedData['16'][test_mergedData['16']=='Found'] = 1
test_mergedData['16'][test_mergedData['16']=='NotFound']=-1

In [ ]:
set(mergedData['15'])
found=(mergedData['15']=='Found')
new=(mergedData['15']=='New')
unknown=(mergedData['15']=='Unknown')
mergedData['15'][found]=-1
mergedData['15'][unknown]=0
mergedData['15'][new]=1
test_found=(test_mergedData['15']=='Found')
test_new=(test_mergedData['15']=='New')
test_unknown=(test_mergedData['15']=='Unknown')
test_mergedData['15'][test_found]=-1
test_mergedData['15'][test_unknown]=0
test_mergedData['15'][test_new]=1

In [ ]:
set(test_mergedData['28'])

{'Found', 'New', nan}

In [ ]:
mergedData['28'][mergedData['28']=='Found'] = -1
mergedData['28'][mergedData['28']=='New']=1
test_mergedData['28'][test_mergedData['28']=='Found'] = -1
test_mergedData['28'][test_mergedData['28']=='New']= 1
#Found=-1 and New=1

In [ ]:
set(test_mergedData['29'])

{'Found', 'NotFound', nan}

In [ ]:
#column 29 is binary valued with the only 2 non-null values being 'Found' and 'Not Found'. Let 'Found' = 1 and 'Not Found' = -1.
mergedData['29'][mergedData['29']=='Found'] = 1
mergedData['29'][mergedData['29']=='NotFound']=-1
test_mergedData['29'][test_mergedData['29']=='Found'] = 1
test_mergedData['29'][test_mergedData['29']=='NotFound']=-1

In [ ]:
mergedData['30'][[0,1,2]]

0    Android 7.0
1     iOS 11.1.2
2            NaN
Name: 30, dtype: object

In [ ]:
set(mergedData['30'])
#android=1
#Mac = 2
#windows=3
#ios=4
#Other =0
androidIdxs=[]
macIdxs=[]
windowsIdxs=[]
iosIdxs=[]
otherIdxs=[]
for i in range(len(mergedData['30'])):
  if (type(mergedData['30'][i])!=str):
    otherIdxs.append(i)
  elif (mergedData['30'][i].find('Android')!=-1):
    androidIdxs.append(i)
  elif (mergedData['30'][i].find('Mac')!=-1):
    macIdxs.append(i)
  elif (mergedData['30'][i].find('Windows')!=-1):
    windowsIdxs.append(i)
  elif (mergedData['30'][i].find('iOS')!=-1):
    iosIdxs.append(i)
  else:
    otherIdxs.append(i)

mergedData['30'][androidIdxs]=1
mergedData['30'][macIdxs]=2
mergedData['30'][windowsIdxs]=3
mergedData['30'][iosIdxs]=4
mergedData['30'][otherIdxs]=0

androidIdxs=[]
macIdxs=[]
windowsIdxs=[]
iosIdxs=[]
otherIdxs=[]
for i in range(len(test_mergedData['30'])):
  if (type(test_mergedData['30'][i])!=str):
    otherIdxs.append(i)
  elif (test_mergedData['30'][i].find('Android')!=-1):
    androidIdxs.append(i)
  elif (test_mergedData['30'][i].find('Mac')!=-1):
    macIdxs.append(i)
  elif (test_mergedData['30'][i].find('Windows')!=-1):
    windowsIdxs.append(i)
  elif (test_mergedData['30'][i].find('iOS')!=-1):
    iosIdxs.append(i)
  else:
    otherIdxs.append(i)

test_mergedData['30'][androidIdxs]=1
test_mergedData['30'][macIdxs]=2
test_mergedData['30'][windowsIdxs]=3
test_mergedData['30'][iosIdxs]=4
test_mergedData['30'][otherIdxs]=0

In [ ]:
set(test_mergedData['34'])

{'match_status:1', 'match_status:2', nan}

In [ ]:
mergedData['34'][mergedData['34']=='match_status:-1']=-1
mergedData['34'][mergedData['34']=='match_status:0']=0
mergedData['34'][mergedData['34']=='match_status:1']=1
mergedData['34'][mergedData['34']=='match_status:2']=2

test_mergedData['34'][mergedData['34']=='match_status:-1']=-1
test_mergedData['34'][mergedData['34']=='match_status:0']=0
test_mergedData['34'][mergedData['34']=='match_status:1']=1
test_mergedData['34'][mergedData['34']=='match_status:2']=2

In [ ]:
nonNumericColNames

['12', '15', '16', '28', '29', '30', '31', '33', '34', 'DeviceInfo']

In [ ]:
mergedDataNonNumericColumns

['12',
 '15',
 '16',
 '28',
 '29',
 '30',
 '31',
 '33',
 '34',
 'DeviceInfo',
 'ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M4']

In [ ]:
mergedDataNullList

[['TransactionId', 0.0],
 ['1', 0.0],
 ['2', 0.0233025729202055],
 ['3', 0.5401607121809849],
 ['4', 0.5401607121809849],
 ['5', 0.05108400990064687],
 ['6', 0.05108400990064687],
 ['9', 0.4805211012736336],
 ['10', 0.4805211012736336],
 ['11', 0.02256765095366525],
 ['12', 0.0],
 ['13', 0.11726165301976663],
 ['14', 0.4450368500967185],
 ['15', 0.022519118370969194],
 ['16', 0.10325653629890524],
 ['17', 0.033723211747658306],
 ['18', 0.6872213709761289],
 ['19', 0.034076806278729554],
 ['20', 0.034472000166397426],
 ['28', 0.02256765095366525],
 ['29', 0.02256765095366525],
 ['30', 0.46222431759722116],
 ['31', 0.027393176318872933],
 ['32', 0.462078719849133],
 ['33', 0.4918707923984109],
 ['34', 0.46056034333335644],
 ['35', 0.022519118370969194],
 ['36', 0.022519118370969194],
 ['37', 0.022519118370969194],
 ['38', 0.022519118370969194],
 ['DeviceType', 0.023732432938370553],
 ['DeviceInfo', 0.17726179168428863],
 ['TransactionID', 0.0],
 ['TransactionDT', 0.0],
 ['TransactionAmt'

In [ ]:
print(mergedData.shape)
test_mergedData.shape

(144233, 402)


(141907, 401)

In [ ]:
#currently, mergedData has 144233 rows with 402 attributes each
over35PercentDataMissing=[]
over35PercentDataMissing.clear()
for i in mergedData.columns:
  if (mergedData[i].isnull().sum()/mergedData.shape[0]) > .35:
    over35PercentDataMissing.append(i)
mergedData=mergedData.drop(columns=over35PercentDataMissing)
test_mergedData=test_mergedData.drop(columns=over35PercentDataMissing)
print(test_mergedData.shape)
print(mergedData.shape)

(141907, 248)
(144233, 249)


In [ ]:
len(over35PercentDataMissing)

153

In [ ]:
#So, this tells us that removing all of the columns in which over 35% of the data is missing will reduce the number of attributes from 402 to 248

In [ ]:

mergedData

,TransactionId,1,2,5,6,11,12,13,15,16,17,19,20,28,29,30,31,35,36,37,38,DeviceType,DeviceInfo,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,...,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,isFraud
0,2987004,0.0,70787.0,NaN,NaN,100.0,-1,NaN,1,-1,166.0,542.0,144.0,1,-1,1,samsung browser 6.2,1,-1,1,1,-1,SAMSUNG SM-G892A Build/NRD90M,2987004,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2987008,-5.0,98945.0,0.0,-5.0,100.0,-1,49.0,1,-1,166.0,621.0,500.0,1,-1,4,mobile safari 11.0,1,-1,-1,1,-1,iOS Device,2987008,86535,15.000,H,2803,100.0,150.0,visa,226.0,debit,anonymous.com,NaN,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2987010,-5.0,191631.0,0.0,0.0,100.0,-1,52.0,-1,1,121.0,410.0,142.0,-1,1,0,chrome 62.0,-1,-1,1,1,1,Windows,2987010,86549,75.887,C,16496,352.0,117.0,mastercard,134.0,credit,gmail.com,gmail.com,1.0,4.0,0.0,1.0,0.0,...,4.0,2.0,2.0,1.0,2.0,2.0,2.0,4.0,4.0,4.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,166.215393,166.215393,166.215393,90.327904,90.327904,31.841299,90.327904,90.327904,90.327904,90.327904,0.000000,0.000000,0.000000,75.887497,75.887497,75.887497,0
3,2987011,-5.0,221832.0,0.0,-6.0,100.0,-1,52.0,1,-1,225.0,176.0,507.0,1,-1,0,chrome 62.0,-1,-1,1,1,1,NaN,2987011,86555,16.495,C,4461,375.0,185.0,mastercard,224.0,debit,hotmail.com,hotmail.com,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,2987016,0.0,7460.0,1.0,0.0,100.0,-1,NaN,-1,1,166.0,529.0,575.0,-1,1,2,chrome 62.0,1,-1,1,1,1,MacOS,2987016,86620,30.000,H,1790,555.0,150.0,visa,226.0,debit,aol.com,NaN,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,100.0,-1,27.0,-1,1,225.0,427.0,139.0,-1,1,0,chrome 66.0 for android,-1,-1,1,-1,-1,F3111 Build/33.3.A.1.97,3577521,15810802,48.877,C,12019,305.0,106.0,mastercard,224.0,debit,yahoo.fr,yahoo.fr,1.0,9.0,0.0,1.0,0.0,...,12.0,1.0,1.0,0.0,1.0,1.0,1.0,9.0,9.0,9.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,548.831970,548.831970,548.831970,60.066002,60.066002,0.000000,60.066002,60.066002,60.066002,60.066002,488.765991,488.765991,488.765991,0.000000,0.000000,0.000000,0
144229,3577526,-5.0,172059.0,1.0,-5.0,100.0,-1,27.0,1,-1,166.0,352.0,333.0,1,-1,1,chrome 55.0 for android,1,-1,1,-1,-1,A574BL Build/NMF26F,3577526,15810876,250.000,R,1214,174.0,150.0,visa,226.0,credit,gmail.com,gmail.com,1.0,1.0,0.0,1.0,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,

In [ ]:
mergedDataNullList.clear()
maxnp = 0.0
for i in mergedData.columns:
  mergedDataNullList.append([i,mergedData[i].isnull().sum()/mergedData.shape[0]])
  if mergedData[i].isnull().sum()/mergedData.shape[0] > maxnp:
    maxnp=mergedData[i].isnull().sum()/mergedData.shape[0]
mergedDataNullList

[['TransactionId', 0.0],
 ['1', 0.0],
 ['2', 0.0233025729202055],
 ['5', 0.05108400990064687],
 ['6', 0.05108400990064687],
 ['11', 0.02256765095366525],
 ['12', 0.0],
 ['13', 0.11726165301976663],
 ['15', 0.022519118370969194],
 ['16', 0.10325653629890524],
 ['17', 0.033723211747658306],
 ['19', 0.034076806278729554],
 ['20', 0.034472000166397426],
 ['28', 0.02256765095366525],
 ['29', 0.02256765095366525],
 ['30', 0.0],
 ['31', 0.027393176318872933],
 ['35', 0.022519118370969194],
 ['36', 0.022519118370969194],
 ['37', 0.022519118370969194],
 ['38', 0.022519118370969194],
 ['DeviceType', 0.023732432938370553],
 ['DeviceInfo', 0.17726179168428863],
 ['TransactionID', 0.0],
 ['TransactionDT', 0.0],
 ['TransactionAmt', 0.0],
 ['ProductCD', 0.0],
 ['card1', 0.0],
 ['card2', 0.006253769941691569],
 ['card3', 0.0011925148891030486],
 ['card4', 0.0012757136022962844],
 ['card5', 0.00662816415106113],
 ['card6', 0.0012341142456996665],
 ['P_emaildomain', 0.09284283069755188],
 ['R_emaildomai

In [ ]:
maxnp
#So, now the highest percentage missing data for a column is 18%

0.17726179168428863

In [ ]:
numericColNames=[]
nonNumericColNames=[]
nonNumericColNames.clear()
numericColNames.clear()
for i in mergedData.columns:
  if (mergedData[i].isnull().sum()/mergedData.shape[0]) != 0.0:
    idx = list(mergedData[i].isnull()).index(False)
    if (type(mergedData[i][idx])==str):
      nonNumericColNames.append(i)
    else:
      numericColNames.append(i)

test_numericColNames=[]
test_nonNumericColNames=[]
test_nonNumericColNames.clear()
test_numericColNames.clear()
for i in test_mergedData.columns:
  if (test_mergedData[i].isnull().sum()/test_mergedData.shape[0]) != 0.0:
    idx = list(test_mergedData[i].isnull()).index(False)
    if (type(test_mergedData[i][idx])==str):
      test_nonNumericColNames.append(i)
    else:
      test_numericColNames.append(i)


In [ ]:
for i in numericColNames:
    mean=mergedData[i].mean()
    nans=mergedData[i].isnull()
    mergedData[i][nans]=mean




In [ ]:
for i in test_numericColNames:
    mean=test_mergedData[i].mean()
    nans=test_mergedData[i].isnull()
    test_mergedData[i][nans]= mean


In [ ]:
for i in nonNumericColNames:
    mode = mergedData[i].mode()[0]
    nans=mergedData[i].isnull()
    mergedData[i][nans]=mode  

for i in test_nonNumericColNames:
    mode = test_mergedData[i].mode()[0]
    nans= test_mergedData[i].isnull()
    test_mergedData[i][nans]=mode  

In [ ]:
ar=[]
for i in test_mergedData.columns:
  ar.append(test_mergedData[i].isnull().sum()/test_mergedData.shape[0])
np.array(ar).argmax()

0

In [ ]:
test_mergedData

,TransactionId,1,2,5,6,11,12,13,15,16,17,19,20,28,29,30,31,35,36,37,38,DeviceType,DeviceInfo,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,...,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,3663586,-45.0,280290.0,0.0,0.0,100.0,-1,27.000000,1,-1,225.0,427.0,563.0,1,-1,0,chrome 67.0 for android,-1,-1,1,-1,-1,MYA-L13 Build/HUAWEIMYA-L13,3663586,18404013,6.271,C,15885,545.0,185.0,visa,138.0,debit,hotmail.com,hotmail.com,1.0,1.0,0.0,1.0,0.0,...,1.243995,1.388985,0.0,0.0,0.0,0.0,0.148695,0.174048,1.0,1.0,1.0,0.0,0.0,0.0,0.291032,0.0,0.0,0.0,0.094817,0.102646,1.0,1.0,1.0,1.0,0.000,0.0000,0.0000,0.0,0.0000,0.0,0.0000,7.869501,14.17575,9.371612,0.0,0.0,0.0,0.000,0.000,0.000
1,3663588,0.0,3579.0,0.0,0.0,100.0,1,36.905715,-1,1,166.0,542.0,368.0,-1,1,1,chrome 67.0 for android,1,-1,1,1,-1,LGLS676 Build/MXB48T,3663588,18404055,50.000,S,2453,399.0,150.0,american express,137.0,credit,gmail.com,gmail.com,4.0,8.0,0.0,4.0,0.0,...,1.000000,3.000000,0.0,7.0,0.0,2.0,1.000000,3.000000,1.0,2.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.0,3.0,2.0,1.0,0.000,270.0000,90.0000,0.0,270.0000,0.0,90.0000,60.000000,310.00000,130.000000,0.0,0.0,0.0,0.000,0.000,0.000
2,3663597,-5.0,185210.0,1.0,0.0,100.0,-1,52.000000,1,-1,225.0,271.0,507.0,1,-1,0,ie 11.0 for tablet,-1,1,1,-1,1,Trident/7.0,3663597,18404235,37.318,C,15885,545.0,185.0,visa,138.0,debit,anonymous.com,anonymous.com,1.0,1.0,0.0,1.0,0.0,...,1.243995,1.388985,0.0,0.0,0.0,0.0,0.148695,0.174048,1.0,1.0,1.0,0.0,0.0,0.0,0.291032,0.0,0.0,0.0,0.094817,0.102646,1.0,1.0,1.0,1.0,0.000,0.0000,0.0000,0.0,0.0000,0.0,0.0000,7.869501,14.17575,9.371612,0.0,0.0,0.0,0.000,0.000,0.000
3,3663601,-45.0,252944.0,0.0,0.0,100.0,-1,27.000000,-1,1,225.0,427.0,563.0,-1,1,0,chrome 67.0 for android,-1,-1,1,-1,-1,MYA-L13 Build/HUAWEIMYA-L13,3663601,18404315,6.271,C,15885,545.0,185.0,visa,138.0,debit,hotmail.com,hotmail.com,2.0,4.0,0.0,3.0,0.0,...,1.243995,1.388985,0.0,0.0,0.0,0.0,0.148695,0.174048,2.0,2.0,2.0,0.0,0.0,0.0,0.291032,1.0,1.0,1.0,0.094817,0.102646,1.0,1.0,1.0,1.0,6.271,6.2710,6.2710,0.0,0.0000,0.0,0.0000,7.869501,14.17575,9.371612,0.0,0.0,0.0,6.271,6.271,6.271
4,3663602,-95.0,328680.0,7.0,-33.0,100.0,-1,27.000000,1,-1,225.0,567.0,507.0,1,-1,0,chrome 67.0 for android,-1,-1,1,-1,-1,SM-G9650 Build/R16NW,3663602,18404359,52.258,C,9633,130.0,185.0,visa,138.0,debit,hotmail.com,hotmail.com,8.0,21.0,0.0,4.0,0.0,...,1.243995,1.388985,0.0,0.0,0.0,0.0,0.148695,0.174048,1.0,1.0,1.0,0.0,0.0,0.0,0.291032,0.0,0.0,0.0,0.094817,0.102646,1.0,1.0,1.0,1.0,0.000,0.0000,0.0000,0.0,0.0000,0.0,0.0000,7.869501,14.17575,9.371612,0.0,0.0,0.0,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141902,4170230,-20.0,473365.0,0.0,0.0,100.0,-1,27.000000,1,-1,225.0,153.0,325.0,1,-1,0,chrome 71.0 for android,-1,-1,1,-1,-1,SM-J700M,4170230,34214253,10.452,C,5812,408.0,185.0,mastercard,224.0,debit,gmail.com,gmail.com,1.0,1.0,0.0,1.0,0.0,...,1.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000,0.000,0.000
141903,4170233,-5.0,489917.0,-4.0,-32.0,100.0,-1,27.000000,-1,1,225.0,417.0,595.0,-1,1,0,chrome 71.0 for android,-1,-1,1,-1,-1,SM-J320M,4170233,34214271,13.403,C,3154,408.0,185.0,mastercard,224.0,debit,hotmail.com,hotmail.com,1.0,2.0,0.0,2.0,0.0,...,0.000000,0.000000,0.0,1.0,0.0,1.0,0.000000,0.000000,1.0,2.0,2.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.

In [ ]:
#Now, our data has no missing values. So, after removing from the mergedData df any columns in which over 35% of the data was missing, I looped through the columns to 
#determine which columns were numeric and which were not (ie: took on string values). For the numeric columns, I replaced the missing values with the mean of that column.
#For the non-numeric columns, I replaced the missing values with the mode (first mode if a tie) of that column. 

In [ ]:
categoricalColumns=[]
nonCategoricalColumns=[]
for i in mergedData.columns:
  if type(mergedData[i].iloc[0])==str:
    categoricalColumns.append(i)
  else:
    nonCategoricalColumns.append(i)
revisedColumnOrder= nonCategoricalColumns + categoricalColumns
mergedData=mergedData[revisedColumnOrder]
revisedColumnOrder.remove('isFraud')
test_mergedData = test_mergedData[revisedColumnOrder]

In [ ]:
fraudVals=mergedData['isFraud'].to_numpy().astype(np.float64)
mergedData = mergedData.drop(columns='isFraud')


In [ ]:
test_mergedData = test_mergedData[mergedData.columns]

In [ ]:
mergedDataOnlyNumericColumns= mergedData.iloc[:,:-7].to_numpy().astype(np.float64)
test_mergedDataOnlyNumericColumns = test_mergedData.iloc[:,:-7].to_numpy().astype(np.float64)

In [ ]:
trainDataNumericOnly, valDataNumericOnly = np.split(mergedDataOnlyNumericColumns, [107000])
trainLabels, valLabels = np.split(fraudVals, [107000])

In [ ]:
valLabels.sum()/valLabels.shape[0]

0.0965004162973706

In [ ]:
trainLabels.sum()/trainLabels.shape[0]

0.072196261682243

In [ ]:
#Splitting the mergedData df into train and val dataframes kept a relatively consistent percentage of the number of transactions that were fraudulent

In [ ]:
trainingData = trainDataNumericOnly
validationData = valDataNumericOnly
testData = test_mergedDataOnlyNumericColumns

In [ ]:
NUM_FEATURES=trainingData.shape[1]

# Simple Network with Acceptable Accuracy

In [ ]:
#The initial model and hyperparameters was inspired by
#https://towardsdatascience.com/pytorch-tabular-binary-classification-a0368da5bb89
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#Standardize features
scaler = StandardScaler()
trainingData = scaler.fit_transform(trainingData)
validationData = scaler.fit_transform(validationData)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 1000
LEARNING_RATE = 0.001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
#check how much trainingdata/validationdata is fraud
print(trainLabels.shape)
print((trainLabels ==1).sum()/trainLabels.shape[0])
print(valLabels.shape)
print((valLabels ==1).sum()/valLabels.shape[0])

(107000,)
0.072196261682243
(37233,)
0.0965004162973706


In [ ]:
#prints accuracy, type 1 error and type 2 error percentage.
#type 1 error is declaring that transaction was fraud but it was not a fraud.
#type 2 error is declaring that transaction was not a fraud but it was a fraud.
def binaryError(y_pred, y_test, epoch):
  problem = 0
  type1 = 0
  type2 = 0
  y_pred_tag = torch.round(torch.sigmoid(y_pred))
  num_fraud = (y_test ==1).sum().float()
  for i in range(len(y_pred)):
    if(y_pred_tag[i] != y_test[i]):
      problem = problem + 1
      if(y_pred_tag[i] == 1):
        type1 = type1 + 1
      else:
        type2 = type2 + 1
  problem = problem / len(y_pred)
  type1 = type1 / (len(y_pred) - num_fraud)
  type2 = type2 / num_fraud
  print(f'Epoch {epoch+0:03}: | Accuracy: {1-problem:.5f}: | Type1 error: {type1:.5f} | Type2 error: {type2:.5f}')



In [ ]:
#Creates index of fraudulent, non-fraudulent cases.
fraudIndex = []
for i in range(len(trainLabels)):
  if (trainLabels[i] == 1):
    fraudIndex.append(i)
fraudIndex = np.asarray(fraudIndex)

fraud_val_index = []
for i in range(len(valLabels)):
  if (valLabels[i] == 1):
    fraud_val_index.append(i)
fraud_val_index = np.asarray(fraud_val_index)

corIndex = []
for i in range(len(trainLabels)):
  if (trainLabels[i] == 0):
    corIndex.append(i)
corIndex = np.asarray(corIndex)


cor_val_index = []
for i in range(len(valLabels)):
  if (valLabels[i] == 0):
    cor_val_index.append(i)
cor_val_index = np.asarray(cor_val_index)


print(corIndex.shape)
print(cor_val_index.size)

(99275,)
33640


In [ ]:
# #This function creates batches with at least 35 percent of the fraud cases.
# both for training and testing
 def batch2(batch_size, training=True, fraudratio_train = 0.35, fraudratio_test = 0.35):
   

   if training:
     fraudSize = int(fraudratio_train * batch_size)
     #random_choice = np.random.choice(trainingData.shape[0], size=batch_size - fraudSize, replace=False)
     random_choice = np.random.choice(corIndex, size = batch_size - fraudSize, replace = True)
     random_fraud = np.random.choice(fraudIndex, size = fraudSize, replace = True)
     random_ind = np.concatenate((random_choice, random_fraud))
     np.random.shuffle(random_ind)  
     input_batch = trainingData[random_ind]
     label_batch = trainLabels[random_ind]
   else:
     fraudSize = int(fraudratio_test * batch_size)
     #random_choice = np.random.choice(validationData.shape[0], size=batch_size - fraudSize, replace=False)

     random_choice = np.random.choice(cor_val_index, size=batch_size - fraudSize, replace=True)
     random_fraud = np.random.choice(fraud_val_index, size = fraudSize, replace = True)
     random_ind = np.concatenate((random_choice, random_fraud))
     np.random.shuffle(random_ind) 
     input_batch = validationData[random_ind]
     label_batch = valLabels[random_ind]
  

   input_batch =  torch.tensor(input_batch).float()
   label_batch =  torch.tensor(label_batch)
  
   return input_batch, label_batch

In [ ]:
#This function prints out the evaluation of a training epoch including 
# Accuracy, type 1 error and type 2 error.
#Note that the validation data has a fixed .1 fraudratio.
def evaluate():
  model.eval()
  test_in, train_label = batch2(37233, training = False, fraudratio_test = 0.1)
  test_in, train_label = test_in.to(device), train_label.to(device)
  output = model(test_in)
  acc = binaryError(output, train_label.unsqueeze(1), epoch)

In [ ]:
#modularized blocks, used these to make hyperparameter tuning easier
def linear_block_with_drop(in_f, out_f, dropout = True, dropoutRate = 0.5):
  return nn.Sequential(
      nn.Linear(in_f, out_f),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.BatchNorm1d(out_f)
  )

def linear_block(in_f, out_f):
  return nn.Sequential(
      nn.Linear(in_f, out_f),
      nn.ReLU(),
      nn.BatchNorm1d(out_f)
  )

def linear_block_no_batchnorm(in_f, out_f):
  return nn.Sequential(
      nn.Linear(in_f, out_f),
      nn.ReLU()
  )

In [ ]:
#The "best" model we achieved through hyperparameter tuning.
#Tested different number of layers, features in each layers,
#Inclusion of dropout and dropout ratio, optimizer, loss and batchnormalization.
#One interesting observation was that if there are more than 5 linear layer, 
#Accuracy drops significantly.
class binaryClassification_modular(nn.Module):
  def __init__(self):
    super(binaryClassification_modular, self).__init__()
    self.encoder = nn.Sequential(
        linear_block(241, 64),
        linear_block(64,64),
        linear_block(64,128),
        linear_block_with_drop(128,128)
    )
    self.layer_out = nn.Linear(128, 1) 

  def forward(self, inputs):
    x = self.encoder(inputs)
    x = self.layer_out(x)
    return x

model = binaryClassification_modular()
print(model)
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


binaryClassification_modular(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=241, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=64, out_features=128, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer_out): Linear(in_features=128, out_features=1, bias=True)
)


In [ ]:
#Evaluation function. Prints accuracy, types of error for each epoch.
for epoch in range(10):
  #accuracy check
  evaluate()

  #actual learning
  for i in range(100):
    epoch_loss = 0
    epoch_acc = 0
    x_data, x_labels = batch2(BATCH_SIZE, fraudratio_train = 0.65)
    x_data, x_labels = x_data.to(device), x_labels.to(device)
    model.zero_grad()
    output = model(x_data)
    loss = criterion(output, x_labels.unsqueeze(1))
    loss.backward()
    optimizer.step()
epoch = 10
evaluate()

Epoch 000: | Accuracy: 0.10182: | Type1 error: 0.99561 | Type2 error: 0.02122
Epoch 001: | Accuracy: 0.72586: | Type1 error: 0.28099 | Type2 error: 0.21246
Epoch 002: | Accuracy: 0.77445: | Type1 error: 0.22265 | Type2 error: 0.25168
Epoch 003: | Accuracy: 0.74504: | Type1 error: 0.25864 | Type2 error: 0.22186
Epoch 004: | Accuracy: 0.79145: | Type1 error: 0.20651 | Type2 error: 0.22697
Epoch 005: | Accuracy: 0.84565: | Type1 error: 0.13915 | Type2 error: 0.29116
Epoch 006: | Accuracy: 0.79969: | Type1 error: 0.19072 | Type2 error: 0.28660
Epoch 007: | Accuracy: 0.79886: | Type1 error: 0.19493 | Type2 error: 0.25705
Epoch 008: | Accuracy: 0.81323: | Type1 error: 0.17690 | Type2 error: 0.27558
Epoch 009: | Accuracy: 0.77077: | Type1 error: 0.22695 | Type2 error: 0.24980
Epoch 010: | Accuracy: 0.79435: | Type1 error: 0.19887 | Type2 error: 0.26672


In [ ]:
#A modified version of binaryError function above.
#This function returns type 1 error and tpye 2 error and also "score"
#That is calculated by type1 error rate + 2 * type2 error rate.
def binaryError_saveData(y_pred, y_test, epoch):
  out = []
  problem = 0
  type1 = 0
  type2 = 0
  y_pred_tag = torch.round(torch.sigmoid(y_pred))
  num_fraud = (y_test ==1).sum().float()
  for i in range(len(y_pred)):
    if(y_pred_tag[i] != y_test[i]):
      problem = problem + 1
      if(y_pred_tag[i] == 1):
        type1 = type1 + 1
      else:
        type2 = type2 + 1
  problem = problem / len(y_pred)
  type1 = type1 / (len(y_pred) - num_fraud)
  type2 = type2 / num_fraud
  print(f'Epoch {epoch+0:03}: | Accuracy: {1-problem:.5f}: | Type1 error: {type1:.5f} | Type2 error: {type2:.5f}')
  out.append(type1)
  out.append(type2)
  out.append(type1 + type2 * 2)
  return out



In [ ]:
#A modified version of evaluate function.
#returns accuracy list with type1,type2 ratio and score.
def evaluate_with_score():
  out = []
  model.eval()
  test_in, train_label = batch2(37233, training = False, fraudratio_test = 0.1)
  test_in, train_label = test_in.to(device), train_label.to(device)
  output = model(test_in)
  acc = binaryError_saveData(output, train_label.unsqueeze(1), epoch)
  return acc

In [ ]:
#This block prints out the epoch with the best score.
#"score" is calculated as type 1 error + 2 * type2 error
prob = 6            #prob * .1 is the probability of fraud rate in training data.
typeError = [0,0]
score_save = 3 #This variable is the initial score. Note that type1 error + 2 * type2 error < 3

for epoch in range(10):
  scoring = evaluate_with_score()
  if (score_save > scoring[2]):
    score_save = scoring[2]
    typeError[0] = scoring[0]
    typeError[1] = scoring[1]
  for i in range(100):
    epoch_loss = 0
    epoch_acc = 0
    x_data, x_labels = batch2(BATCH_SIZE, fraudratio_train = prob * .1)
    x_data, x_labels = x_data.to(device), x_labels.to(device)
    model.zero_grad()
    output = model(x_data)
    loss = criterion(output, x_labels.unsqueeze(1))
    loss.backward()
    optimizer.step()
print(f'Probability {prob * 0.1:.1f}: | Type1 error: {typeError[0]:.5f} | Type2 error: {typeError[1]:.5f}')


Epoch 000: | Accuracy: 0.79247: | Type1 error: 0.20152 | Type2 error: 0.26162
Epoch 001: | Accuracy: 0.81009: | Type1 error: 0.18027 | Type2 error: 0.27666
Epoch 002: | Accuracy: 0.83539: | Type1 error: 0.14942 | Type2 error: 0.30137
Epoch 003: | Accuracy: 0.79918: | Type1 error: 0.19281 | Type2 error: 0.27290
Epoch 004: | Accuracy: 0.83117: | Type1 error: 0.15434 | Type2 error: 0.29922
Epoch 005: | Accuracy: 0.79040: | Type1 error: 0.20507 | Type2 error: 0.25034
Epoch 006: | Accuracy: 0.78138: | Type1 error: 0.21316 | Type2 error: 0.26779
Epoch 007: | Accuracy: 0.77230: | Type1 error: 0.22450 | Type2 error: 0.25651
Epoch 008: | Accuracy: 0.81978: | Type1 error: 0.16762 | Type2 error: 0.29358
Epoch 009: | Accuracy: 0.80359: | Type1 error: 0.18535 | Type2 error: 0.29600
Probability 0.6: | Type1 error: 0.20507 | Type2 error: 0.25034


# Simple Neural Network

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import imblearn
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):        
        super().__init__()
        # TODO: Filling in Model Layers, these are just placeholders
        self.L1      = torch.nn.Linear(NUM_FEATURES, 150)
        self.L2      = torch.nn.Linear(150, 100)
        #self.L3      = torch.nn.Linear(100,50)
        #self.L4      = torch.nn.Linear(50,10)
        #self.L5      = torch.nn.Linear(10,1)
        self.L5      = torch.nn.Linear(100,1)
        self.ReLU    = torch.nn.ReLU()
        self.Sigmoid = torch.nn.Sigmoid()
        self.Dropout = torch.nn.Dropout(p=0.3)
        
    def forward(self, x):
        ''' forward pass '''
        # TODO: Implement Model Layers in Forward Pass
        x = self.ReLU(self.L1(x))
        x = self.ReLU(self.L2(x))
        #x = self.ReLU(self.L3(x))
        #x = self.ReLU(self.L4(x))
        x = self.L5(x)

        return x

    def predict(self, x):
        ''' predict forecast '''
        # TODO: Predict Forecast Given Data
        pass

In [ ]:
#index of fraudulent cases
fraudIndex = []
for i in range(len(trainLabels)):
  if (trainLabels[i] == 1):
    fraudIndex.append(i)
fraudIndex = np.asarray(fraudIndex)


In [ ]:
# #This function creates batches with 30 percent of the fraud cases. Original function is located below.
 def batch2(batch_size, training=True):
#   """Create a batch of examples.
#   #
#   This creates a batch of inputs and a batch of corresponding
#   ground-truth labels.
  
#   Args:
#     batch_size: An integer.
#     training: A boolean. If True, grab examples from the training
#       set; otherwise, grab them from the validation set.
  
#   Returns:
#     A tuple,
#     input_batch: A Tensor of floats with shape
#       [batch_size, num_features]
#     label_batch: A Tensor of ints with shape
#       [batch_size].
#   """
   if training:
     fraudRatio = 0.35
     fraudSize = int(fraudRatio * batch_size)
     random_choice = np.random.choice(trainingData.shape[0], size=batch_size - fraudSize, replace=False)
     random_fraud = np.random.choice(fraudIndex, size = fraudSize, replace = False)
     random_ind = np.concatenate((random_choice, random_fraud))
     np.random.shuffle(random_ind)
     input_batch = trainingData[random_ind]
     label_batch = trainLabels[random_ind]
   else:
     input_batch = validationData[:batch_size]
     label_batch = valLabels[:batch_size]
  
#   # Create PyTorch Tensors from our NumPy arrays
   # TODO: Fill in each line.
   input_batch =  torch.tensor(input_batch, requires_grad=False)
   label_batch =  torch.tensor(label_batch, requires_grad=False)
  
   return input_batch, label_batch

In [ ]:
def batch(batch_size, training=True):
  if training:
    random_ind = np.random.choice(trainingData.shape[0], size=batch_size, replace=False)
    input_batch = trainingData[random_ind]
    label_batch = trainLabels[random_ind]
  else:
    input_batch = validationData[:batch_size]
    label_batch = valLabels[:batch_size]
  
#   # Create PyTorch Tensors from our NumPy arrays
#   # TODO: Fill in each line.
  input_batch =  torch.tensor(input_batch, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  def batch(batch_size, training=True):
  if training:
    random_ind = np.random.choice(trainingData.shape[0], size=batch_size, replace=False)
    input_batch = trainingData[random_ind]
    label_batch = trainLabels[random_ind]
  else:
    input_batch = validationData[:batch_size]
    label_batch = valLabels[:batch_size]
  
#   # Create PyTorch Tensors from our NumPy arrays
#   # TODO: Fill in each line.
  input_batch =  torch.tensor(input_batch, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  
  return input_batch, label_batch
  return input_batch, label_batch

IndentationError: ignored

In [ ]:
#draws batches from equally balanced classes that are derived from random under sampling module

def randomUnderSamplingBatch(batch_size):
  from imblearn.under_sampling import RandomUnderSampler
  rus = RandomUnderSampler(sampling_strategy='majority')
  trainUS, trainLabelsUS = rus.fit_sample(trainingData, trainLabels)
  random_choice = np.random.choice(trainUS.shape[0], size=batch_size, replace=False)
  np.random.shuffle(random_choice)
  input_batch = trainUS[random_choice]
  label_batch = trainLabelsUS[random_choice]
  input_batch =  torch.tensor(input_batch, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  return input_batch, label_batch

In [ ]:
#Creates batches and perform's resampling by removing Tomek's links

def removeTomeksLinks(batch_size):
  from imblearn.under_sampling import TomekLinks
  sampler = TomekLinks()
  dataTL, labelTL = sampler.fit_sample(trainingData, trainLabels)
  random_choice = np.random.choice(dataTL.shape[0], size=batch_size, replace=False)
  np.random.shuffle(random_choice)
  input_batch = dataTL[random_choice]
  label_batch = labelTL[random_choice]
  input_batch =  torch.tensor(input_batch, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  return input_batch, label_batch

In [ ]:
#Creates batches that samples from equally balanced classes dervied from SMOTE algorithm

def smoteOverSampling(batch_size):
  from imblearn.over_sampling import SMOTE
  sampler = SMOTE(sampling_strategy='minority')
  trainSmote, labelSmote = sampler.fit_sample(trainingData, trainLabels)
  random_choice = np.random.choice(trainSmote.shape[0], size=batch_size, replace=False)
  np.random.shuffle(random_choice)
  input_batch = trainSmote[random_choice]
  label_batch = labelSmote[random_choice]
  input_batch =  torch.tensor(input_batch, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  return input_batch, label_batch


In [ ]:
def accuracy(pred, true):
  correct = (true.eq(pred)).sum()  / true.nelement()

In [ ]:
#Training Parameters, We can Tweak These
batch_size  = 3000
epochs      = 150
learning_rate = 0.001

In [ ]:
model = Net()
# TODO: choose a loss function, We'll use BCE
loss_fn = torch.nn.BCEWithLogitsLoss(reduction='mean')
# TODO: choose an optimizer, SGD or ADAM. SGD allows for control of stuff like weight decay and momentum
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
''' Training Loop USING BATCH2 function WHICH ENSURES FRAUDS ARE AT LEAST 35% OF DATA  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
fraudBatchIdxs=[]
recall=0
for epoch in range(epochs):
    fraudBatchIdxs.clear()
    model.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = batch2(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, yes, no).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall*1.0/epochs)

0.603
0.625
0.6593333333333333
0.6946666666666667
0.7186666666666667
0.7326666666666667
0.731
0.7456666666666667
0.756
0.7536666666666667
0.7546666666666667
0.76
0.7623333333333333
0.777
0.767
0.773
0.7723333333333333
0.7653333333333333
0.7756666666666666
0.774
0.7776666666666666
0.7696666666666667
0.787
0.792
0.7813333333333333
0.7866666666666666
0.7906666666666666
0.8016666666666666
0.7996666666666666
0.7773333333333333
0.7963333333333333
0.799
0.8016666666666666
0.795
0.8143333333333334
0.8106666666666666
0.815
0.8143333333333334
0.808
0.8033333333333333
0.8143333333333334
0.81
0.8093333333333333
0.7996666666666666
0.8116666666666666
0.8163333333333334
0.824
0.8163333333333334
0.8056666666666666
0.8196666666666667
0.8143333333333334
0.8193333333333334
0.8156666666666667
0.8196666666666667
0.823
0.8176666666666667
0.822
0.817
0.8243333333333334
0.8266666666666667
0.824
0.831
0.8263333333333334
0.8233333333333334
0.8176666666666667
0.833
0.823
0.833
0.83
0.8243333333333334
0.838666666

In [ ]:
''' Training Loop with use OF BATCH FUNCTION  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
fraudBatchIdxs=[]
recall = 0
for epoch in range(epochs):
    fraudBatchIdxs.clear()
    model.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = batch(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, yes, no).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall/epochs*1.0)

NameError: ignored

In [ ]:
''' Training Loop USING RANDOM UNDER SAMPLING BATCH function WHICH ENSURES FRAUDS ARE AT LEAST 50% OF DATA  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
recall=0
for epoch in range(epochs):
    
    model.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = randomUnderSamplingBatch(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, torch.tensor(1), torch.tensor(0)).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/pred.size()[0]
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall*1.0/epochs)

0.8203333333333334
0.8433333333333334
0.8433333333333334
0.861
0.854
0.8616666666666667
0.8566666666666667
0.8503333333333334
0.8336666666666667
0.8423333333333334
0.8363333333333334
0.8473333333333334
0.8486666666666667
0.8546666666666667
0.858
0.8523333333333334
0.8466666666666667
0.8486666666666667
0.8453333333333334
0.8373333333333334
0.8593333333333333
0.8513333333333334
0.8426666666666667
0.8523333333333334
0.8643333333333333
0.851
0.8556666666666667
0.8533333333333334
0.8566666666666667
0.856
0.8416666666666667
0.8506666666666667
0.8633333333333333
0.8623333333333333
0.862
0.861
0.8626666666666667
0.8503333333333334
0.857
0.86
0.852
0.8523333333333334
0.859
0.861
0.8596666666666667
0.864
0.8556666666666667
0.862
0.857
0.87
0.8746666666666667
0.866
0.8676666666666667
0.858
0.8446666666666667
0.869
0.861
0.8656666666666667
0.8566666666666667
0.853
0.8706666666666667
0.8573333333333333
0.8566666666666667
0.8536666666666667
0.862
0.8676666666666667
0.8806666666666667
0.879
0.859
0.8

In [ ]:
''' Training Loop USING batch function that removes Tomek's Links  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []

recall=0
for epoch in range(25):
    fraudBatchIdxs.clear()
    model.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = removeTomeksLinks(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, torch.tensor(1), torch.tensor(0)).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall*1.0/epochs)

In [ ]:
''' Training Loop USING batch function that oversamples using SMOTE algo  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []

recall=0
for epoch in range(epochs):
    model.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = smoteOverSampling(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, torch.tensor(1), torch.tensor(0)).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Loss[epa])
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall*1.0/epochs)

In [ ]:
#forcing the batches to have at 50% frauds caused the model to have a higher average recall percentage than the 35% fraud or a completely random batch scheme (56 to 47% to 11%)

In [ ]:
class Net2(torch.nn.Module):
    def __init__(self):        
        super().__init__()
        self.L1      = torch.nn.Linear(NUM_FEATURES, NUM_FEATURES)
        self.L2      = torch.nn.Linear(NUM_FEATURES, 150)
        self.L3      = torch.nn.Linear(150,100)
        self.L4      = torch.nn.Linear(100,100)
        self.L5      = torch.nn.Linear(100,50)
        self.L6      = torch.nn.Linear(50,1)
        self.ReLU    = torch.nn.ReLU()
        self.Dropout = torch.nn.Dropout(p=0.3)
        
    def forward(self, x):
        ''' forward pass '''
        x = self.ReLU(self.L1(x))
        x = self.ReLU(self.L2(x))
        x = self.ReLU(self.L3(x))
        x = self.ReLU(self.L4(x))
        x = self.ReLU(self.L5(x))
        x = self.L6(x)

        return x

    def predict(self, x):
        ''' predict forecast '''
        # TODO: Predict Forecast Given Data
        pass

In [ ]:
model2 = Net2() #Using deeper NN here
loss_fn = torch.nn.BCEWithLogitsLoss(reduction='mean', pos_weight=torch.tensor(3))
# TODO: choose an optimizer, SGD or ADAM. SGD allows for control of stuff like weight decay and momentum
#optimizer = torch.optim.SGD(model2.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

In [ ]:
''' Training Loop with use OF BATCH2 FUNCTION AND NET2 NN  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
recall = 0
for epoch in range(epochs):
    model2.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = batch2(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model2(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, yes, no).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    print()
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall/epochs*1.0)

In [ ]:
valyes = torch.ones(valLabels.shape[0], 1)
valno = torch.zeros(valLabels.shape[0], 1)
val = model2.forward(torch.tensor(validationData, requires_grad=False, dtype=torch.float))
pred = torch.where(val >= 0.5, valyes, valno).squeeze().detach().numpy()
val_acc = ((pred == valLabels).sum() / valLabels.shape[0])
val_acc

In [ ]:
#NOTE: IN THIS MODEL, I SET POS_WEIGHT PARAMETER OF THE POSITIVE EXAMPLE TO BE 3 AND WE ACHIEVE CLOSE TO 70% RECALL

In [ ]:
''' Training Loop with use OF random under sampling FUNCTION AND NET2 NN  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
recall = 0
for epoch in range(epochs):
    model2.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = randomUnderSamplingBatch(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model2(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, yes, no).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall/epochs*1.0)

In [ ]:
''' Training Loop with use OF SMTOE FUNCTION AND NET2 NN  '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
recall = 0
for epoch in range(epochs):
    model2.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, output_batch = smoteOverSampling(batch_size)
    output_batch = output_batch.float()
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, NUM_FEATURES)
    output = model2(input_batch.float())
    loss = loss_fn(output.squeeze(), output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred = torch.where(output >= 0.5, yes, no).squeeze()
    torchTwos = torch.ones(output_batch.size()[0])*2
    actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred==output_batch).sum().item()/batch_size
    Train_Accuracy.append(tracc)
    print(Train_Accuracy[epoch])
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))
print(recall/epochs*1.0)

NameError: ignored

In [ ]:
#We notice that the most important columns to identify users are D10n, D1n, D15n, C13, D4n, card1, D2n, card2, addr1, TransactionAmt, and dist1. These are the columns we must use to find the clients.



##NN With Word Embeddings

In [ ]:
 train_identity = pd.read_csv("/content/drive/My Drive/train_identity.csv")

In [ ]:
colNames=['TransactionId','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30',
          '31','32','33','34','35','36','37','38','DeviceType','DeviceInfo']
train_identity.columns=colNames
nullList=[['TransactionId',0.0]]
for i in range(38):
  j=i+1
  idx=str(j)
  perecentageNull=(train_identity[idx].isnull().sum())/train_identity.shape[0]
  nullList.append([i+1,perecentageNull])

In [ ]:
nullList.append(['DeviceType',train_identity['DeviceType'].isnull().sum()/train_identity.shape[0]])
nullList.append(['DeviceInfo',train_identity['DeviceInfo'].isnull().sum()/train_identity.shape[0]])
nullList

In [ ]:
Cols2Delete=[]
for i in range(len(nullList)-3):
  j=i+1
  if (nullList[j][1]>0.9):
    Cols2Delete.append(str(j))

In [ ]:
train_identity=train_identity.drop(columns=Cols2Delete)

In [ ]:
nonNumericColNames=[]
for i in train_identity.columns:
  j=0
  notFound=True
  idxNull=train_identity[i].isnull()
  while (j<len(train_identity[i])) and notFound:
    if idxNull[j]==False:
      notFound=False
      if (type(train_identity[i][j])==str):
        nonNumericColNames.append(i)
    j+=1

In [ ]:
for i in [35,36,37,38]:
  idxsTrue=(train_identity[str(i)]=='T')
  idxsFalse=(train_identity[str(i)]=='F')
  train_identity[str(i)][idxsTrue] = 1
  train_identity[str(i)][idxsFalse] = -1


In [ ]:
 train_transaction = pd.read_csv("/content/drive/My Drive/train_transaction.csv")

In [ ]:
transactionNullList=[]
for i in train_transaction.columns:
  percentageNull = train_transaction[i].isnull().sum()/train_transaction.shape[0]
  transactionNullList.append([i,perecentageNull])

In [ ]:
mergedData=train_identity.merge(train_transaction,left_on=['TransactionId'],right_on=['TransactionID'])

In [ ]:
columns2remove=[]
for i in mergedData.columns:
  if ((mergedData[i].isnull().sum()/mergedData.shape[0])>0.75):
    columns2remove.append(i)
mergedData=mergedData.drop(columns=columns2remove)

In [ ]:
colNamesMergedData=mergedData.columns
colNamesMergedData=list(colNamesMergedData)
isFraudIndex=colNamesMergedData.index('isFraud')
colNamesMergedData = colNamesMergedData[:isFraudIndex] + colNamesMergedData[isFraudIndex+1:]
colNamesMergedData.append('isFraud')
mergedData=mergedData[colNamesMergedData]

In [ ]:
mergedDataNullList=[]
mergedDataNullList.clear()
for i in mergedData.columns:
  percentNull = mergedData[i].isnull().sum()/mergedData.shape[0]
  mergedDataNullList.append([i,percentNull])

In [ ]:
over35PercentDataMissing=[]
over35PercentDataMissing.clear()
for i in mergedData.columns:
  if (mergedData[i].isnull().sum()/mergedData.shape[0]) > .35:
    over35PercentDataMissing.append(i)
mergedData=mergedData.drop(columns=over35PercentDataMissing)

In [ ]:
numericColNames=[]
nonNumericColNames=[]
nonNumericColNames.clear()
numericColNames.clear()
for i in mergedData.columns:
  if (mergedData[i].isnull().sum()/mergedData.shape[0]) != 0.0:
    idx = list(mergedData[i].isnull()).index(False)
    if (type(mergedData[i][idx])==str):
      nonNumericColNames.append(i)
    else:
      numericColNames.append(i)

In [ ]:
for i in numericColNames:
  mean=mergedData[i].mean()
  nans=mergedData[i].isnull()
  mergedData[i][nans]=mean

In [ ]:
for i in nonNumericColNames:
  mode = mergedData[i].mode()[0]
  nans=mergedData[i].isnull()
  mergedData[i][nans]=mode  

In [ ]:
categoricalColumns=[]
nonCategoricalColumns=[]
for i in mergedData.columns:
  if type(mergedData[i].iloc[0])==str:
    categoricalColumns.append(i)
  else:
    nonCategoricalColumns.append(i)
revisedColumnOrder= nonCategoricalColumns + categoricalColumns
mergedData=mergedData[revisedColumnOrder]

In [ ]:
fraudVals=mergedData['isFraud'].to_numpy().astype(np.float64)
mergedData=mergedData.drop(columns='isFraud')

In [ ]:
trainData, valData = np.split(mergedData, [107000])
trainLabels, valLabels = np.split(fraudVals, [107000])

In [ ]:
trainData

In [ ]:
#set(trainData['31'])

In [ ]:
'''for i in range(len(trainData['31'])):
  string = trainData['31'][i]
  string = string.split()[0]
  string = string.split('/')[0]
  trainData['31'][i]=string'''

In [ ]:
#trainData.drop(columns=['DeviceInfo'])

In [ ]:
'''for i in ['12','15','16','28','29','31']:
  if (trainData[i]=='NotFound').sum()!=0:
     trainData[i][trainData[i]=='NotFound']='not found'
  if (trainData[i]=='Found').sum()!=0:
     trainData[i][trainData[i]=='Found']='found'
  if ((trainData[i]=='New').sum()!=0):
    trainData[i][trainData[i]=='New']='new'''''

In [ ]:
#trainData=trainData.drop(columns=['DeviceInfo'])

In [ ]:
'''for i in ['P_emaildomain','R_emaildomain']:
  for j in range(trainData.shape[0]):
    string = trainData[i][j]
    string=string.split('.')[0]
    trainData[i][j]=string
trainData'''

In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1999)
clf.fit(trainingData, trainLabels)

In [ ]:
pred = clf.predict(validationData)
score = sum(pred == valLabels)/valLabels.size
score

## NN with Word Embeddings

##new data set



In [ ]:
import torch
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import skimage.data
import skimage.color
import scipy.misc
import scipy.signal
import time
import torch.nn.functional as F
from matplotlib import rcParams
rcParams['axes.grid'] = False
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/My Drive/creditcard.csv") #read in data after importing libraries and mounting drive

In [ ]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [ ]:
data.shape

(284807, 31)

In [ ]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [ ]:
#no missing data

In [ ]:
data['Class'].sum()/data.shape[0]

0.001727485630620034

In [ ]:
#only .1% of data is fraudulent

In [ ]:
data.corrwith(data['Class'])

In [ ]:
#let's split into training and test set before we oversample the training data 
from sklearn.model_selection import train_test_split
trainData, testData = train_test_split(data, test_size=0.25, shuffle=True)



In [ ]:
trainData.shape

(213605, 31)

In [ ]:
testData.shape

(71202, 31)

In [ ]:
testLabels=np.asarray(testData['Class']).reshape(-1)
testData=np.asarray(testData.drop(columns=['Class'])).reshape(testData.shape[0],-1)

In [ ]:
#so we have split our data into a train set and a test set. the test set is 25% of the original data and it has been further split into the test data and the labels. both of which are arrays

In [ ]:
trainLabels=trainData['Class']
trainData=trainData.drop(columns=['Class'])

In [ ]:
#now we will oversample the minority class by using the smote algorithm

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
trainData, trainLabels = smote.fit_sample(trainData, trainLabels)


In [ ]:
#trainData and train labels are now both resampled datasets

In [ ]:
trainData.shape

(426454, 30)

In [ ]:
trainLabels.shape

(426454,)

In [ ]:
#smoteData contains oversampled data wrt to the smote algorithm and smoteLabels contains the corresponding labels

In [ ]:
trainLabels.sum()/trainLabels.shape[0]

0.5

In [ ]:
#let's start building a nn

In [ ]:
testLabels=testLabels.squeeze()

In [ ]:
# either randomly samples from the train set or randomly samples from the test set 
def batch(batch_size, training=True):
  if training:
    random_ind = np.random.choice(trainData.shape[0], size=batch_size, replace=False)
    input_batch = trainData[random_ind]
    label_batch = trainLabels[random_ind]
  else:
    random_ind = np.random.choice(testData.shape[0], size=batch_size, replace=False)
    input_batch = testData[random_ind]
    label_batch = testLabels[random_ind]
  
#   # Create PyTorch Tensors from our NumPy arrays

  input_batch =  torch.tensor(input_batch, dtype=torch.float, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
numfeatures=trainData.shape[1]

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import imblearn
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
class model1(torch.nn.Module):
    def __init__(self):        
        super().__init__()
        self.L1      = torch.nn.Linear(numfeatures, numfeatures)
        self.L2      = torch.nn.Linear(numfeatures, 20)
        self.L3      = torch.nn.Linear(20,10)
        self.L4      = torch.nn.Linear(10,2)
        self.ReLU    = torch.nn.ReLU()
        self.Dropout = torch.nn.Dropout(p=0.3)
        
    def forward(self, x):
        ''' forward pass '''
        x = self.Dropout(self.ReLU(self.L1(x)))
        x = self.ReLU(self.L2(x))
        x = self.ReLU(self.L3(x))
        x = self.L4(x)
        return x
#returns a [N,1,2] shaped tensor

In [ ]:
#instantiate the model
modelOne = model1()
batch_size=300

In [ ]:
#instantiate the optimizer 
optimizer = torch.optim.Adam(modelOne.parameters(), lr=0.001)

In [ ]:
''' Loop with use OF model one on training data that is randomly sampled each pass    '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
from sklearn.metrics import confusion_matrix
pred=[]
avgCM=np.zeros((2,2))
recall = 0
for epoch in range(300):
    pred=[]
    modelOne.train()
    input_batch, output_batch = batch(batch_size)
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, numfeatures)
    output = modelOne(input_batch.float()).squeeze()
    loss = F.cross_entropy(output, output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred=[]
    for i in range(output.shape[0]):
      if (output[i][0]>output[i][1]):
        pred.append(0)
      else:
        pred.append(1)
    pred=np.asarray(pred).reshape(len(pred))
    #actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    #recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred == output_batch.numpy()).sum()/batch_size
    Train_Accuracy.append(tracc)
    avgCM+=(confusion_matrix(output_batch.numpy(), pred))
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(avgCM/300)
modelOne.eval()
input_batch, output_batch = batch(batch_size, training=False)
input_batch = input_batch.reshape(-1, numfeatures)
output = modelOne(input_batch.float()).squeeze()
pred=[]
for i in range(output.shape[0]):
  if (output[i][0]>output[i][1]):
    pred.append(0)
  else:
    pred.append(1)
pred=np.asarray(pred).reshape(len(pred))
print((pred == output_batch.numpy()).sum()/batch_size)

[[79.79666667 70.90333333]
 [79.20666667 70.09333333]]
0.0


In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import imblearn
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
class model2(torch.nn.Module):
    def __init__(self):        
        super().__init__()  
        self.L1     = torch.nn.Linear(numfeatures, 15)
        self.L2     = torch.nn.Linear(15,2)
        self.ReLU    = torch.nn.ReLU()
        self.Dropout = torch.nn.Dropout(p=0.3)
        
    def forward(self, x):
        ''' forward pass '''
        x = self.ReLU(self.L1(x))
        x = self.L2(x)
        return x
#returns a [N,1,2] shaped tensor

In [ ]:
#instantiate the model
modelTwo = model2()
batch_size=300

In [ ]:
#instantiate the optimizer 
optimizer = torch.optim.Adam(modelOne.parameters(), lr=0.001)

In [ ]:
''' Loop with use OF model one on training data that is randomly sampled each pass    '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
from sklearn.metrics import confusion_matrix
pred=[]
avgCM=np.zeros((2,2))
recall = 0
for epoch in range(300):
    pred=[]
    modelTwo.train()
    input_batch, output_batch = batch(batch_size)
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, numfeatures)
    output = modelTwo(input_batch.float()).squeeze()
    loss = F.cross_entropy(output, output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred=[]
    for i in range(output.shape[0]):
      if (output[i][0]>output[i][1]):
        pred.append(0)
      else:
        pred.append(1)
    pred=np.asarray(pred).reshape(len(pred))
    #actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    #recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred == output_batch.numpy()).sum()/batch_size
    Train_Accuracy.append(tracc)
    avgCM+=(confusion_matrix(output_batch.numpy(), pred))
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(avgCM/300)
modelOne.eval()
input_batch, output_batch = batch(batch_size, training=False)
input_batch = input_batch.reshape(-1, numfeatures)
output = modelTwo(input_batch.float()).squeeze()
pred=[]
for i in range(output.shape[0]):
  if (output[i][0]>output[i][1]):
    pred.append(0)
  else:
    pred.append(1)
pred=np.asarray(pred).reshape(len(pred))
print((pred == output_batch.numpy()).sum()/batch_size)

[[  0.         150.25666667]
 [  0.         149.74333333]]
0.0


In [ ]:
##NO SMOTE

In [ ]:
data = pd.read_csv("/content/drive/My Drive/creditcard.csv") #read in data after importing libraries and mounting drive

In [ ]:
data.shape

(284807, 31)

In [ ]:
#let's split into training and test set before we oversample the training data 
from sklearn.model_selection import train_test_split
trainData, testData = train_test_split(data, test_size=0.25, shuffle=True)

In [ ]:
testLabels=np.asarray(testData['Class']).reshape(-1)
testData=np.asarray(testData.drop(columns=['Class'])).reshape(testData.shape[0],-1)

In [ ]:
trainLabels=np.asarray(trainData['Class']).reshape(-1)
trainData = np.asarray(trainData.drop(columns=['Class'])).reshape(trainData.shape[0],-1)

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(sampling_strategy=float(1/9))
trainData, trainLabels = ros.fit_resample(trainData, trainLabels)

In [ ]:
# either randomly samples from the train set or randomly samples from the test set 
def batch(batch_size, training=True):
  if training:
    random_ind = np.random.choice(trainData.shape[0], size=batch_size, replace=False)
    input_batch = trainData[random_ind]
    label_batch = trainLabels[random_ind]
  else:
    random_ind = np.random.choice(testData.shape[0], size=batch_size, replace=False)
    input_batch = testData[random_ind]
    label_batch = testLabels[random_ind]
  
#   # Create PyTorch Tensors from our NumPy arrays

  input_batch =  torch.tensor(input_batch, dtype=torch.float, requires_grad=False)
  label_batch =  torch.tensor(label_batch, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
#train on model one again except without the use of smote  BUT does use weights in the cross entropy loss function and does oversample to make the minority class 1% of the dataset

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import imblearn
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
class model1(torch.nn.Module):
    def __init__(self):        
        super().__init__()
        self.L1      = torch.nn.Linear(numfeatures, numfeatures)
        self.L2      = torch.nn.Linear(numfeatures, 20)
        self.L3      = torch.nn.Linear(20,10)
        self.L4      = torch.nn.Linear(10,2)
        self.ReLU    = torch.nn.ReLU()
        self.Dropout = torch.nn.Dropout(p=0.3)
        
    def forward(self, x):
        ''' forward pass '''
        x = self.Dropout(self.ReLU(self.L1(x)))
        x = self.ReLU(self.L2(x))
        x = self.ReLU(self.L3(x))
        x = self.L4(x)
        return x
#returns a [N,1,2] shaped tensor

In [ ]:
#instantiate the model
modelOne = model1()
batch_size=10000

In [ ]:
#instantiate the optimizer 
optimizer = torch.optim.Adam(modelOne.parameters(), lr=0.005)

In [ ]:
''' Loop with use OF model one on training data that is randomly sampled each pass    '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
yes = torch.ones(batch_size, 1)
no = torch.zeros(batch_size, 1)
from sklearn.metrics import confusion_matrix
pred=[]
avgCM=np.zeros((2,2))
recall = 0

for epoch in range(250):
    pred=[]
    modelOne.train()
    input_batch, output_batch = batch(batch_size)
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, numfeatures)
    output = modelOne(input_batch.float()).squeeze()
    lossfn = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.1,0.90]))
    loss = lossfn(output, output_batch) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred=[]
    for i in range(output.shape[0]):
      if (output[i][0]>output[i][1]):
        pred.append(0)
      else:
        pred.append(1)
    pred=np.asarray(pred).reshape(len(pred))
    #actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    #recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred == output_batch.numpy()).sum()/batch_size
    Train_Accuracy.append(tracc)
    avgCM+=(confusion_matrix(output_batch.numpy(), pred))
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(avgCM/250)


[[5166.932 3832.472]
 [ 567.084  433.512]]


tensor(10)

In [ ]:
#false positive problem!!!

##Autoencoder


In [ ]:
data = pd.read_csv("/content/drive/My Drive/creditcard.csv")

In [ ]:
#re-read in the data

In [ ]:
from sklearn import model_selection
train, test = model_selection.train_test_split(data, test_size=0.25)

In [ ]:
#will only train on the non-fradulent cases of the train data so throw out the frauds

In [ ]:
train=train.query('Class==0')
train=train.reset_index(drop=True)
length = train.shape[0]
index=pd.Series(data=np.arange(0,length))
train.set_index(index)
length = test.shape[0]
index= pd.Series(data=np.arange(0,length))
test.set_index(index)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,129892.0,1.873381,0.465134,-1.269965,3.699792,0.756307,-0.076264,0.500784,-0.077093,-1.214551,1.658832,-0.078736,-0.216768,-1.331819,0.812526,-1.927469,0.656039,-0.806325,-0.060104,-0.869978,-0.304083,0.016308,-0.016584,0.029393,-0.385072,0.184357,0.020927,-0.079617,-0.070353,37.93,0
1,710.0,1.408027,-0.331867,-0.313791,-0.849112,-0.361621,-0.863772,-0.076457,-0.283323,-1.494481,0.830413,1.288182,0.326140,0.779445,0.157626,-0.326645,0.989968,-0.003279,-1.078421,1.042453,0.154036,0.234797,0.584994,-0.235836,0.072392,0.826615,-0.085643,-0.030822,-0.008092,25.00,0
2,55589.0,-0.703697,-0.404724,1.271373,-2.383789,-0.549200,-1.368112,0.244533,-0.210514,-2.591891,0.906959,-0.606036,-1.087350,0.303274,-0.247422,-0.265098,-0.423602,0.373301,-0.456782,-0.543556,-0.016308,-0.472680,-1.179169,0.214850,0.299886,-0.027200,-0.664821,0.288750,0.175040,64.94,0
3,44419.0,-1.850197,-0.059949,2.246166,0.986533,-0.305455,1.461981,-0.108060,-0.358717,-0.113765,-0.280938,1.399852,0.795906,-0.123954,-0.085980,0.752591,-0.424797,0.227166,0.082218,0.074765,-0.492250,0.934730,0.790787,0.175939,-0.263111,0.288114,-0.058318,0.264711,0.031978,156.09,0
4,74149.0,-1.748086,-0.745209,0.063610,-1.874769,3.312007,2.646876,-0.082670,0.832836,-0.510660,-0.689444,-0.195347,-0.246205,-0.082273,0.258462,0.699013,0.742693,-1.008519,-0.093083,-0.002003,0.231804,-0.177206,-0.993750,0.050614,1.031276,0.414286,0.736295,-0.275616,0.255039,83.48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71197,54324.0,-0.104975,-1.788651,1.281619,3.151649,-0.939291,2.527630,-0.524582,0.754029,0.521000,-0.038987,-0.066793,0.633496,-0.410463,-0.472764,0.136741,-0.894335,1.171708,-2.070393,-2.535468,0.698122,0.417508,0.477521,-0.300209,-0.601864,-0.183348,0.136044,0.038400,0.113912,511.89,0
71198,56330.0,1.050827,-0.178368,1.013529,1.502354,-0.698471,0.459080,-0.551032,0.367703,0.766997,-0.055872,0.837398,0.825683,-1.458204,0.041971,-1.295890,-0.629982,0.327480,-0.321730,-0.037168,-0.257605,-0.029859,0.184924,-0.049359,0.216778,0.516543,-0.284417,0.061348,0.012894,13.24,0
71199,65594.0,-0.563965,1.216241,2.346170,2.073634,0.072976,-0.171311,0.792260,-0.262718,-0.635812,0.504349,-0.533629,0.365584,0.617310,-0.667691,-0.912776,0.008852,-0.325892,-0.758311,-0.539691,-0.041772,-0.219910,-0.446971,0.069864,0.682718,-0.714869,-0.465726,-0.269859,-0.037933,7.57,0
71200,31772.0,-0.610858,0.841679,1.383381,0.128666,0.431677,-0.603175,0.928020,0.100537,-0.392727,-0.620820,-0.671738,-0.956380,-2.169941,0.758280,0.605942,0.114163,-0.288828,-0.799582,-1.063219,-0.281191,-0.339290,-1.220904,0.208146,-0.070215,-0.541049,-1.006836,0.110001,0.143350,23.73,0


In [ ]:
train['Class'].sum()

0

In [ ]:
#all the frauds in the train data have been thrown out so now we have no need for the class column

In [ ]:
train=train.drop(columns=['Class'])

In [ ]:
train.shape[1]

30

In [ ]:
#autoencoder design: 30 -> 28 -> 25 -> 20 -> 25 -> 28 -> 30

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import imblearn
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
class autoencoder(torch.nn.Module):
    def __init__(self):        
        super().__init__()
        self.L1      = torch.nn.Linear(30, 28)
        self.L2      = torch.nn.Linear(28, 25)
        self.L3      = torch.nn.Linear(25,20)
        self.L4      = torch.nn.Linear(20,25)
        self.L5      = torch.nn.Linear(25,28)
        self.L6      = torch.nn.Linear(28,30)
        self.ReLU    = torch.nn.ReLU()
        self.Dropout = torch.nn.Dropout(p=0.3)
        
    def forward(self, x):
        ''' forward pass '''
        x = self.ReLU(self.L1(x))
        x = self.ReLU(self.L2(x))
        x = self.ReLU(self.L3(x))
        x = self.ReLU(self.L4(x))
        x = self.ReLU(self.L5(x))
        x = self.L6(x)
        return x
#returns a [N,1,30] shaped tensor

In [ ]:
#  randomly samples from the train set
def batchAutoencoder(batch_size):
  random_ind = list(np.random.choice(train.shape[0], size=batch_size, replace=False))
  input_batch = train.iloc[random_ind].to_numpy().astype(np.double)
  label_batch = train.iloc[random_ind].to_numpy().astype(np.double)
  # Create PyTorch Tensors from our NumPy arrays
  input_batch =  torch.tensor(input_batch, dtype=torch.double, requires_grad=False)
  label_batch =  torch.tensor(label_batch, dtype=torch.double, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
autoen = autoencoder()
batch_size=500

In [ ]:
#instantiate the optimizer 
optimizer = torch.optim.Adam(autoen.parameters(), lr=0.001)

In [ ]:
''' train autoencoder on training data that is randomly sampled each pass    '''
totalloss=0
elementLTotal=0
for epoch in range(1000):
    autoen.train()
    # TODO: implement training on your network, compute the loss, update weights
    inputs, orig = batchAutoencoder(batch_size)
    orig = orig.reshape(-1,30)
    optimizer.zero_grad()
    inputs= inputs.reshape(-1, 30)
    encoded = autoen(inputs.float()).squeeze()
    for i in range(batch_size):
      elementLTotal+=torch.median((torch.abs(encoded[i]-orig[i])))
    loss_fn = torch.nn.MSELoss()
    loss=loss_fn(encoded.double(), orig.double())
    totalloss+=loss.item()
    loss.backward()
    optimizer.step()
avgAbsMedianElementErr=elementLTotal/(500*1000)
print(avgAbsMedianElementErr)

tensor(9.4848, dtype=torch.float64, grad_fn=<DivBackward0>)


In [ ]:
#autoencoder is now trained on the train data 

In [ ]:
#we now want to set some threshold and pass the test data into the autoencoder; if the error we are analyzing is greater than the threshold, we classify
# the transaction as fraud. otherwise, we classify it to be a normal transaction 

In [ ]:
test=test.set_index(pd.Series(data=np.arange(0,test.shape[0])))
testLabels=test['Class']
testData=test
testData=testData.drop(columns=['Class'])

In [ ]:
testData=testData.to_numpy().astype(np.double)
testData

array([[ 1.29892000e+05,  1.87338082e+00,  4.65133687e-01, ...,
        -7.96174201e-02, -7.03533906e-02,  3.79300000e+01],
       [ 7.10000000e+02,  1.40802734e+00, -3.31866826e-01, ...,
        -3.08215235e-02, -8.09235626e-03,  2.50000000e+01],
       [ 5.55890000e+04, -7.03696800e-01, -4.04723596e-01, ...,
         2.88750053e-01,  1.75039699e-01,  6.49400000e+01],
       ...,
       [ 6.55940000e+04, -5.63964616e-01,  1.21624072e+00, ...,
        -2.69858922e-01, -3.79325831e-02,  7.57000000e+00],
       [ 3.17720000e+04, -6.10857666e-01,  8.41679010e-01, ...,
         1.10001338e-01,  1.43350203e-01,  2.37300000e+01],
       [ 3.36280000e+04, -1.00065523e+00,  8.51782400e-01, ...,
         2.00146219e-01,  9.77634060e-02,  1.00000000e+00]])

In [ ]:
testLabels=testLabels.to_numpy().astype(np.double)

In [ ]:
#  batch function for detecting frauds using the autoencoder
def batchAnomaly(batch_size):
  random_ind = list(np.random.choice(testData.shape[0], size=batch_size, replace=False))
  input_batch = testData[random_ind].astype(np.double)
  label_batch = testLabels[random_ind].astype(np.double)
  # Create PyTorch Tensors from our NumPy arrays
  input_batch =  torch.tensor(input_batch, dtype=torch.double, requires_grad=False)
  label_batch =  torch.tensor(label_batch, dtype=torch.double, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
'''
from sklearn.metrics import confusion_matrix
for epoch in range(300):
    # implement the autoencoder on the batch from the test set to find anomalies
    autoen.eval()
    inputs, labels = batchAnomaly(batch_size)
    inputs= inputs.reshape(-1, 30)
    outputs = autoen(inputs.float()).squeeze()
    pred=[]
    for i in range(batch_size):
      absMedErr=torch.median(torch.abs(outputs[i]-inputs[i]))
      if (absMedErr > 2*avgAbsMedianElementErr):
        pred.append(1) #classify transaction as fraud
      else:
        pred.append(0) #classify transaction as normal 
    pred = np.asarray(pred).reshape(-1)
    labels=labels.numpy().reshape(-1)
    print(confusion_matrix(labels,pred))
    '''


'\nfrom sklearn.metrics import confusion_matrix\nfor epoch in range(300):\n    # implement the autoencoder on the batch from the test set to find anomalies\n    autoen.eval()\n    inputs, labels = batchAnomaly(batch_size)\n    inputs= inputs.reshape(-1, 30)\n    outputs = autoen(inputs.float()).squeeze()\n    pred=[]\n    for i in range(batch_size):\n      absMedErr=torch.median(torch.abs(outputs[i]-inputs[i]))\n      if (absMedErr > 2*avgAbsMedianElementErr):\n        pred.append(1) #classify transaction as fraud\n      else:\n        pred.append(0) #classify transaction as normal \n    pred = np.asarray(pred).reshape(-1)\n    labels=labels.numpy().reshape(-1)\n    print(confusion_matrix(labels,pred))\n    '

In [ ]:
testLabels.shape

(71202,)

In [ ]:
frauds=[]
for i in range(testLabels.shape[0]):
  if testLabels[i]==1:
    frauds.append(i)
frauds

[717,
 995,
 1515,
 3365,
 3833,
 4023,
 5164,
 5370,
 5668,
 7173,
 7885,
 7918,
 7929,
 8940,
 9993,
 10051,
 10732,
 12726,
 13868,
 14002,
 14849,
 15293,
 16374,
 17415,
 17718,
 17770,
 17967,
 18862,
 19850,
 19980,
 20248,
 20648,
 21556,
 21607,
 24572,
 25073,
 25217,
 25354,
 27332,
 27587,
 27673,
 29312,
 29533,
 29827,
 29925,
 30915,
 31881,
 31973,
 32212,
 34222,
 34454,
 34764,
 35037,
 35323,
 35389,
 35460,
 35633,
 36032,
 36167,
 36206,
 36267,
 36530,
 36643,
 36906,
 37283,
 37778,
 37874,
 38972,
 39707,
 40138,
 42216,
 43097,
 44048,
 44387,
 44391,
 44680,
 46259,
 46416,
 46559,
 47404,
 47714,
 48257,
 48776,
 48917,
 49330,
 49516,
 49596,
 50993,
 51108,
 51616,
 52191,
 55530,
 55536,
 56241,
 56255,
 56385,
 56574,
 56611,
 57422,
 58354,
 58456,
 58766,
 58788,
 60129,
 60443,
 60827,
 60864,
 61736,
 62046,
 62407,
 63082,
 66108,
 66430,
 70271,
 70566]

In [ ]:
len(frauds)

115

In [ ]:
testLabels[[frauds[0],frauds[1]]]

189701    1
154668    1
Name: Class, dtype: int64

In [ ]:
testData

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
179698,124203.0,1.847404,-1.185214,-1.091277,-0.689695,-0.044272,1.263598,-0.800935,0.295787,-0.560313,0.820916,0.872712,1.306591,1.369511,-0.045403,0.161004,-1.899347,0.147073,0.207574,-1.417636,-0.424874,-0.209718,-0.045228,0.203935,-1.599408,-0.532189,0.658079,0.004063,-0.061423,95.90
87268,61607.0,-0.523557,0.419637,0.521676,-2.205197,1.021148,0.411423,1.051972,0.214867,0.320478,-1.804641,0.727641,1.257002,0.537520,0.418545,1.589079,-1.798652,0.695392,-2.485828,-1.505691,-0.183362,-0.063195,-0.073153,0.316542,-0.993255,-0.829123,-1.194567,0.246902,0.180463,51.93
73256,55055.0,-4.581092,-4.571681,1.110058,2.907234,0.492698,0.018268,-0.601009,1.092282,0.260432,-0.805733,0.254634,1.311827,-0.579389,0.100612,-1.821185,-1.187199,1.148000,-0.147569,1.406739,2.247379,0.360652,-0.817214,1.141933,-0.024318,0.091315,-0.439262,0.203688,-0.454215,522.98
84636,60397.0,-1.665781,-1.144142,3.010905,-0.610786,-0.317435,1.324789,-0.685626,0.854277,-0.147971,-0.571158,-0.019369,0.527467,-1.354157,-0.847886,-3.076387,-1.931564,0.414161,0.664694,-2.420267,-0.443295,-0.298404,-0.433379,0.114989,-0.371256,0.426139,-0.520229,0.075684,0.072809,104.50
48694,43745.0,-0.246418,1.507680,1.367568,2.517219,-0.345926,0.429856,-0.576798,-2.331898,-1.358382,0.959398,1.501731,0.990880,0.380280,0.581608,0.189282,0.537014,-0.333580,0.074642,0.122260,-0.429988,2.125896,-1.337926,0.275977,0.468193,0.483079,-0.094455,0.238044,0.237868,8.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75255,55971.0,-1.620154,-0.867839,1.469044,1.370479,-0.771379,0.685584,0.850804,0.138521,0.209900,-0.360742,0.118353,-0.061682,-1.540383,0.026937,-0.806631,-0.291070,-0.027334,0.816949,1.275026,0.092676,0.090729,0.334595,0.080396,-0.000183,0.130065,-0.216929,0.096412,0.207816,353.10
13800,24466.0,-0.886470,-0.126264,3.551005,3.751230,-0.563369,1.141458,-0.889331,0.234025,2.022399,0.262257,-0.746190,-2.832099,0.993826,0.036043,-1.809902,-0.697789,1.335534,0.634633,1.106144,-0.046956,-0.058542,0.915166,0.376580,0.375107,-0.396255,0.368435,0.028081,-0.129019,3.80
262053,160290.0,-1.047060,0.664435,0.669760,-0.542247,0.489718,-1.114482,0.511222,0.296111,-0.564560,-1.076502,-1.081523,0.024624,-0.281243,0.538521,-0.168314,0.013486,-0.095475,-0.719187,-0.006107,-0.094435,-0.231529,-0.980237,-0.206872,-0.090383,0.601474,0.330412,-0.252606,-0.212934,6.28
145175,86690.0,2.066310,0.141414,-1.871986,0.181612,0.767652,-0.263277,0.116804,-0.095535,0.168725,-0.227865,0.808499,1.022278,0.717027,-0.806692,-0.531346,0.623649,0.009212,0.265858,0.520653,-0.086922,-0.337165,-0.868154,0.259410,0.014492,-0.201522,0.182048,-0.062927,-0.044107,1.98


In [ ]:
#  batch function for detecting frauds using the autoencoder I WILL INTENTIONALLY PACK FRAUDS INTO THIS BATCH
def batchAnomaly1():
  random_ind = list(np.random.choice(testData.shape[0], size=500, replace=False))
  input_batch = testData[random_ind].astype(np.double)
  label_batch = testLabels[random_ind].astype(np.double)
  input_batch = np.concatenate((input_batch,testData[frauds].astype(np.double)))
  label_batch = np.concatenate((label_batch, testLabels[frauds].astype(np.double)))
  # Create PyTorch Tensors from our NumPy arrays
  input_batch =  torch.tensor(input_batch, dtype=torch.double, requires_grad=False)
  label_batch =  torch.tensor(label_batch, dtype=torch.double, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
from sklearn.metrics import confusion_matrix
totalCM=np.zeros((2,2))
for epoch in range(500):
    # implement the autoencoder on the batch from the test set to find anomalies
    autoen.eval()
    inputs, labels = batchAnomaly1()
    inputs= inputs.reshape(-1, 30)
    outputs = autoen(inputs.float()).squeeze()
    pred=[]
    for i in range(outputs.shape[0]):
      absMedErr=torch.median(torch.abs(outputs[i]-inputs[i]))
      if (absMedErr > 0.3 * avgAbsMedianElementErr):
        pred.append(1) #classify transaction as fraud
      else:
        pred.append(0) #classify transaction as normal 
    pred = np.asarray(pred).reshape(-1)
    labels=labels.numpy().reshape(-1)
    totalCM+=(confusion_matrix(labels,pred))
print(totalCM/500)

[[330.68  168.474]
 [ 38.276  77.57 ]]


In [ ]:
#  batch function for detecting frauds using the autoencoder I WILL INTENTIONALLY PACK FRAUDS INTO THIS BATCH
def batchAnomaly2():
  random_ind = list(np.random.choice(testData.shape[0], size=testData.shape[0], replace=False))
  input_batch = testData[random_ind].astype(np.double)
  label_batch = testLabels[random_ind].astype(np.double)
  # Create PyTorch Tensors from our NumPy arrays
  input_batch =  torch.tensor(input_batch, dtype=torch.double, requires_grad=False)
  label_batch =  torch.tensor(label_batch, dtype=torch.double, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
from sklearn.metrics import confusion_matrix

# implement the autoencoder on the batch from the test set to find anomalies
autoen.eval()
inputs, labels = batchAnomaly2()
inputs= inputs.reshape(-1, 30)
outputs = autoen(inputs.float()).squeeze()
pred=[]
for i in range(outputs.shape[0]):
  absMedErr=torch.median(torch.abs(outputs[i]-inputs[i]))
  if (absMedErr > 0.25 * avgAbsMedianElementErr):
    pred.append(1) #classify transaction as fraud
  else:
    pred.append(0) #classify transaction as normal 
pred = np.asarray(pred).reshape(-1)
labels=labels.numpy().reshape(-1)
print(confusion_matrix(labels,pred))


[[35007 36080]
 [   23    92]]


##Dataset 3 (UCSD)


In [ ]:
labels = pd.read_csv("/content/drive/My Drive/2009Task2TrainLabels.csv")

In [ ]:
data = pd.read_csv("/content/drive/My Drive/2009Task2TrainInputs.csv")

In [ ]:
data

In [ ]:
labels.columns=['isFraud']


In [ ]:
labels

In [ ]:
d1={'isFraud':0}
labels=labels.append(d1,ignore_index=True)
labels

In [ ]:
labels.shape[0]==data.shape[0]

In [ ]:
data=pd.concat([data, labels['isFraud']],axis=1)

In [ ]:
data

In [ ]:
#now data is the dataframe of the UCSD credit card dataset that has the class labels in the isFraud category

In [ ]:
data.isnull().sum()

In [ ]:
#one row has one null value in the state1 attr

In [ ]:
data.iloc[list(data['state1'].isnull()).index(True)]

In [ ]:
data=data.drop([80876])

In [ ]:
data=data.set_index(np.arange(0,data.shape[0]))

In [ ]:
data.iloc[80876]

In [ ]:
#now data is a dataframe with no missing data whatsoever

In [ ]:
data.shape

In [ ]:
#99999 x 20 df
categoricalCount=0
for i in range(data.shape[1]):
  if type(data.loc[0][i])==str:
    categoricalCount+=1
categoricalCount


In [ ]:
data

In [ ]:
#only categorical columns are state1 and custAtrr2 attributes

In [ ]:
len(set(data['custAttr1'])) #unique values in custAttr1

In [ ]:
len(set(data['custAttr2'])) #unique values in custAttr2 

In [ ]:
uniqueEmails=list(set(data['custAttr2']))
emailIdxs={}
for i in range(len(uniqueEmails)):
  emailIdxs[uniqueEmails[i]]=(data['custAttr2']==uniqueEmails[i])
  if (i%1000==0):
    print(i)
emailsAndFraud={}
for i in range(len(uniqueEmails)):
  subset=data[emailIdxs.get(uniqueEmails[i])]
  emailsAndFraud[uniqueEmails[i]]=subset['isFraud']/subset.shape[0]
  if(i%1000==0):
    print(i)

In [ ]:
data

In [ ]:
#through quick eda it doesnt not seem that the emails nor custAttr1 give away fraud
data.corrwith(data['isFraud'])

In [ ]:
#remember, there are only 2 categorical attributes; these attributes are the state values and the emails. since there are only 50 possible values for a state and the abbreviations of the state offers no real information
#other than a way to differentiate states from one another, it is fine to convert the states into values from 1-50. i will deal with the emails in the same way. there is no intuitive notion of closeness or similarity
# between the values of these categorical attributes so converting them to numbers doesn't lose much

In [ ]:
stateDict={}
count=1
for i in range(data.shape[0]):
  state=str(data['state1'][i])
  if state not in stateDict.keys():
    stateDict[state]=count
    count+=1
  if (i%1000==0):
    print(i)

In [ ]:
for i in range(data.shape[0]):
  data['state1'][i]=stateDict.get(str(data['state1'][i]))
data

In [ ]:
data.isnull().sum()

In [ ]:
#now let's do the same thing with custAttr2

In [ ]:
emailDict={}
count=1
for i in range(data.shape[0]):
  email=str(data['custAttr2'][i])
  if email not in emailDict.keys():
    emailDict[email]=count
    count+=1
  if (i%1000==0):
    print(i)

In [ ]:
for i in range(data.shape[0]):
  data['custAttr2'][i]=emailDict.get(str(data['custAttr2'][i]))
data

In [ ]:
#all numerical data; no entry/attr missing
#can build model now

from sklearn import model_selection
train, test = model_selection.train_test_split(data, test_size=0.3)
testLabels = test['isFraud']
testData = test.drop(columns=['isFraud'])
from imblearn.over_sampling import SMOTE
sampler = SMOTE(sampling_strategy=0.25)
trainLabels=train['isFraud']
trainData= train.drop(columns=['isFraud'])
trainData, trainLabels = sampler.fit_sample(trainData, trainLabels)


In [ ]:
trainData.shape

In [ ]:
trainLabels.sum()/trainLabels.shape[0]

In [ ]:
#separated data into test set and train set. train set has been oversampled wrt to the smote algorithm 

In [ ]:
batch_size=500

In [ ]:
#  randomly samples from the train set
def batchUCSD(batch_size):
  random_ind = np.random.choice(trainData.shape[0], size=batch_size, replace=False)
  input_batch = trainData[random_ind].astype(np.double)
  label_batch = trainLabels[random_ind].astype(np.long)
  # Create PyTorch Tensors from our NumPy arrays
  input_batch =  torch.tensor(input_batch, dtype=torch.double, requires_grad=False)
  label_batch =  torch.tensor(label_batch, dtype=torch.long, requires_grad=False)
  
  return input_batch, label_batch

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import imblearn
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
class fcNN(torch.nn.Module):
    def __init__(self):        
        super().__init__()
        # TODO: Filling in Model Layers, these are just placeholders
        self.L1      = torch.nn.Linear(19, 19)
        self.L2      = torch.nn.Linear(19, 15)
        self.L3      = torch.nn.Linear(15,10)
        self.L4      = torch.nn.Linear(10,10)
        self.L5      = torch.nn.Linear(10,5)
        self.L6      = torch.nn.Linear(5,2)
        self.ReLU    = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.1)
        
    def forward(self, x):
        ''' forward pass '''
        x = self.ReLU(self.L1(x))
        x = self.ReLU(self.L2(x))
        x = self.ReLU(self.L3(x))
        x = self.ReLU(self.L4(x))
        x = self.ReLU(self.L5(x))
        x = self.L6(x)
        return x.squeeze()
#returns a [N,2] shaped tensor

In [ ]:
fcnn=fcNN()

In [ ]:
optimizer = torch.optim.Adam(fcnn.parameters(), lr=0.01)

In [ ]:
''' Loop with use OF model fcnn on training data that is randomly sampled each pass    '''
Loss = []
Train_Accuracy = []
Test_Accuracy  = []
from sklearn.metrics import confusion_matrix
recall = 0
for epoch in range(300):
    fcnn.train()
    # TODO: implement training on your network, compute the loss, update weights
    input_batch, y = batchUCSD(batch_size)
    optimizer.zero_grad()
    input_batch = input_batch.reshape(-1, 19)
    output = fcnn(input_batch.float())
    lossfn = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.25,0.75]))
    loss = lossfn(output, y) 
    loss.backward()
    optimizer.step()
    Loss.append(loss)
    pred=[]
    for i in range(output.shape[0]):
      if (output[i][0]>output[i][1]):
        pred.append(0)
      else:
        pred.append(1)
    pred=np.asarray(pred).reshape(len(pred))
    #actualFrauds=torch.where(output_batch==1,output_batch, torchTwos)
    #recall+=(pred==actualFrauds).sum().item()/((actualFrauds==1).sum().item())
    tracc = (pred == y.numpy()).sum()/batch_size
    Train_Accuracy.append(tracc)
    print(confusion_matrix(y.numpy(), pred))
    #The follow below code also evaluates the model for ALL test data. Its commented out since
    #this code most likely won't work at beginning, but the general logic is there.
    """
    model.eval()
    toutput = model(test_x)
    teacc = accuracy(toutput, test_y)
    Test_Accuracy.append(teacc)
    """
print(max(Train_Accuracy))

In [ ]:
#max accuracy with dropout p = 0.3 was .592
#when reduced layer with dropout, max accuracy was marginally better at .604